In [1]:
import chemicals
import chemics as cm
from iapws import IAPWS97
import scipy
import pandas as pd
import pint
ureg = pint.UnitRegistry()
from chemlib import empirical_formula_by_percent_comp as efbpc

In [2]:
def delete_ash_water(fuel):
    print('Eliminando el agua y las cenizas de la composición porcentual del combustible...')
    ash_water = {}
    for element in fuel:
        if element == 'H2O':
            ash_water['H2O'] = fuel['H2O']
        if element == 'Ash':
            ash_water['Ash'] = fuel['Ash']
    new_fuel = fuel.copy()
    try:
        del new_fuel['Ash']
        del new_fuel['H2O']
        del new_fuel['Cl']
    except:
        pass
    correction_factor = (100 - sum(ash_water.values()))/100
    print('Aplicando el factor de corrección...')
    for element in new_fuel:
        new_fuel[element] = fuel[element] / correction_factor
    print('La composición del combustible ahora es:')
    print(new_fuel)
    return new_fuel

In [3]:
def elemental_composition(fuel, elemental_comp = True):
    print('Obteniendo composición molecular...')
    peso_molecular = {}
    print('Transformando composición porcentual a composición elemental a traves del peso molecular...')
    for element in fuel.keys():
        if element == 'Ash':
            peso_molecular[element] = 0
        else:
            if elemental_comp == True:

                peso_molecular[element] = fuel[element]/cm.mw(element)
            else:
                peso_molecular[element] = fuel[element]
    return peso_molecular

In [4]:
def get_reactants_and_products(fuel, relacion_O2N2):
    react = fuel
    sol_s = chemicals.combustion_stoichiometry(fuel)
    print(sol_s)
    prods = {}
    for key in react:
        react[key] =round(react[key], 3)
    for element in sol_s:
        if sol_s[element] < 0:
            react[element] = - round(sol_s[element],3)
        else:
            prods[element] = round(sol_s[element],3)
    react['N2'] = round(relacion_O2N2 * abs(sol_s['O2']),3)
    prods['N2'] = round(relacion_O2N2 * abs(sol_s['O2']),3)
    print('Los reactantes son:')
    print(react)
    print('Los productos son:')
    print(prods)
    return react, prods

In [5]:
def get_excess_air(reactantes, productos, lambda_air):
    productos['N2'] = productos['N2'] * lambda_air
    productos['O2'] = reactantes['O2'] * 2 * lambda_air - productos['CO2'] * 2 - productos['H2O']
    reactantes['N2'] = reactantes['N2'] * lambda_air
    reactantes['O2'] = reactantes['O2'] * lambda_air
    print('Con exceso de aire...')
    print('Los reactantes son:')
    print(reactantes)
    print('Los productos son:')
    print(productos)
    return reactantes, productos
#lambda_air = 1.3

In [6]:
def get_fuel(fuel_name, file = 'coal_composition.csv'):
    df = pd.read_csv(file)
    elements = df.columns.tolist()[1:9]
    composition = df[df['fuel_type'] == fuel_name][elements].values.flatten().tolist()
    create_fuel = dict(zip(elements, composition))
    return create_fuel

In [7]:
def get_mass_molecular(react, prods):
    aire = {}
    combustible = {}
    productos_masa = {}
    for element in react:
        if element == 'O2' or element == 'N2':
            aire[element]  = react[element] * cm.mw(element)
        else:
            combustible[element] = react[element] * cm.mw(element)
    for element in prods:
        productos_masa[element] = prods[element] * cm.mw(element)
    productos_masa = {k: v for (k, v) in productos_masa.items() if v >= 0}
    print('La masa de combustible es de: ', (round(sum(combustible.values()),2)),'[kg]')
    print('La masa de aire es de: ', (round(sum(aire.values()),2)),'[kg]')
    print('La masa de los productos es de: ', productos_masa)
    print('La suma de masa de los productos es de: ', (round(sum(productos_masa.values()))),'[kg]')
    suma_reactantes = (round(sum(combustible.values()))) + (round(sum(aire.values())))
    suma_productos = (round(sum(productos_masa.values())))
    diferencia = suma_reactantes - suma_productos
    if diferencia < 1:
        print('Se comprueba la conservación de masa')
    else:
        print('Existe una diferencia de ', diferencia, 'en el balance de masa')
    return combustible, aire, productos_masa

In [8]:
def get_rac(air, fuel, products): #másico o molar
    sum_air = (round(sum(air.values())))
    sum_fuel = (round(sum(fuel.values())))
    rac = sum_air / sum_fuel #kg aire / kg combustible
    gases_humedos = (round(sum(products.values()))) / sum_fuel
    gases_secos = ((round(sum(products.values())))  - products['H2O']) / sum_fuel
    print('La relación Aire / Combustible (RAC) másico es:', rac)
    print('La relación Gases húmedos / Combustible es:', gases_humedos)
    print('La relación Gases secos / Combustible es:', gases_secos)
    return rac, gases_secos, gases_humedos

In [9]:
#Comparación con la EPA
#fname = 'natural_gas'
#fmass_flow = 1
def epa_flow(fuel_type, fuel_mass_flow_kghour, PCS_MJkg, lambda_aire):
    df = pd.read_csv('coal_composition.csv').dropna()
    if fuel_type in (df['fuel_type'].tolist()):
        fd = df[df['fuel_type'] == fuel_type]['fd'].item() * ureg.cubic_feet / ureg.british_thermal_unit
        fw = df[df['fuel_type'] == fuel_type]['fw'].item() * ureg.cubic_feet / ureg.british_thermal_unit
        fc = df[df['fuel_type'] == fuel_type]['fc'].item() * ureg.cubic_feet / ureg.british_thermal_unit
    else:
        corrector = (0.947817 / 2.2046216) * 1000
        #Dry list
        fd_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O']]
        fd_factors = [3.64,1.53,0.57,0.14,-0.46]
        fd = [a * b for a,b in zip(fd_list,fd_factors)]
        fd = sum(fd) * 10**6 / (PCS_MJkg.magnitude * corrector) * ureg.cubic_feet / ureg.british_thermal_unit

        #Wet list
        fw_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O'], fuel_wet['H2O']]
        fw_factors = [5.56,1.53,0.575,0.14,-0.46,0.21]
        fw = [a * b for a,b in zip(fw_list,fw_factors)]
        fw = sum(fw) * 10**6 / (PCS_MJkg.magnitude * corrector) * ureg.cubic_feet / ureg.british_thermal_unit

        fc = fw
    print('El factor fd es:', fd)
    print('El factor fw es:', fw)
    print('El factor fc es:', fc)
    PCS = PCS_MJkg * 947.817 * ureg.british_thermal_unit / ureg.megajoule
    #print(PCS)
    fuel_mass_wu = fuel_mass_flow_kghour
    #print(fuel_mass_wu)
    gas_flow_dry = fuel_mass_wu * PCS * fd / 10**6 * lambda_aire
    gas_flow_wet = fuel_mass_wu * PCS * fw / 10**6 * lambda_aire
    gas_flow_co2 = fuel_mass_wu * PCS * fc / 10**6 * lambda_aire

    print('El flujo de gases secos según EPA es:', gas_flow_dry)
    print('El flujo de gases húmedos según EPA es:', gas_flow_wet)
    print('El flujo de gases CO" según EPA es:', gas_flow_co2)

    return gas_flow_dry, gas_flow_wet, gas_flow_co2


In [10]:
def analytic_flow(products_mass, fuel_mass, fuel_mass_flow, gases_secos_flow, gases_humedos_flow):
    peso_molecular_gases = (cm.mw('O2') * aire_O2 + cm.mw('N2') * aire_N2) / 100
    P_est = scipy.constants.physical_constants['standard atmosphere'][0]
    rho_gas = cm.gas_density.rhog(peso_molecular_gases, P_est, 293.15)
    co2_mass_analytic = products_mass['CO2']
    co2_density = cm.gas_density.rhog(cm.mw('CO2'), P_est, 293.15)
    fuel_mass_analytic = sum(fuel_mass.values())
    mass_gas_co2 = (co2_mass_analytic / fuel_mass_analytic)
    #print(rho_gas)
    #print(co2_density)
    #print(fuel_mass_flow)
    #print(gases_secos_flow)
    vol_gases_dry = gases_secos_flow / rho_gas * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    vol_gases_wet = gases_humedos_flow / rho_gas * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    vol_gases_co2 = mass_gas_co2 / co2_density * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    print('El flujo de gases secos análiticos es:', vol_gases_dry)
    print('El flujo de gases húmedos análiticos es:', vol_gases_wet)
    print('El flujo de gases CO" análiticos es:', vol_gases_co2)
    return vol_gases_dry, vol_gases_wet, vol_gases_co2

In [11]:
def EPA_analysis(epa, analytic):
    EPA_dry, EPA_wet, EPA_CO2 = [abs(epa[i]- analytic[i]) / epa[i]*100 for i in range(len(epa))]
    print('El porcentaje de error entre el valor análitico y el valor EPA son los siguientes:')
    print('Flujo seco:', round(EPA_dry.magnitude,2), '%')
    print('Flujo húmedo:', round(EPA_wet.magnitude,2), '%')
    print('Flujo CO2:', round(EPA_CO2.magnitude,2), '%')
    return EPA_dry, EPA_wet, EPA_CO2

In [12]:
#Composición aire
aire_O2 = 21
aire_N2 = 79
razon_N2O2 = aire_N2 / aire_O2

In [13]:
156/cm.mw('O2')
32.07*295.82/3600 * 0.4 #mcomb*PCS/3600 #MWh
#2.6 #MWh
#1/2.17

1.0541052666666666

In [14]:
fmass_flow = 99.53 * ureg.kilogram / ureg.hour
fmass_flow.magnitude

99.53

In [15]:
df = pd.read_csv('coal_composition.csv').dropna()
fuel = 'bituminous_coal'
if fuel in (df['fuel_type'].tolist()):
    print('yeah')

yeah


In [16]:
PCS = df[df['fuel_type'] == fuel]['PCS_MJ/KG'].item()

In [17]:
print('Análisis estequiométrico')
df = pd.read_csv('coal_composition.csv')
fuel = ' vcjubituminous_coal'
#fmass_flow = 99.53 * ureg.kilogram / ureg.hour
PCS = df[df['fuel_type'] == fuel]['PCS_MJ/KG'].item() * ureg.megajoule / ureg.kilogram
lambda_air = 1
fuel_wet = get_fuel(fuel)
fuel_dry = delete_ash_water(fuel_wet)
elem_comp = elemental_composition(fuel_dry, elemental_comp=True)
reactivos, productos = get_reactants_and_products(elem_comp, razon_N2O2)
fuel_mass, air_mass, products_mass = get_mass_molecular(reactivos, productos)
fmass_flow = sum(fuel_mass.values()) * ureg.kilogram / ureg.hour
rac_mass, gases_secos, gases_humedos = get_rac(air_mass, fuel_mass, products_mass)
epa_flows = epa_flow(fuel, fmass_flow, PCS, lambda_air)
analytic_flows = analytic_flow(products_mass, fuel_mass, fmass_flow, gases_secos, gases_humedos)
analysis = EPA_analysis(epa_flows, analytic_flows)

Análisis estequiométrico


ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
print('Análisis estequiométrico')
df = pd.read_csv('coal_composition.csv')
fuel = 'bituminous_coal'
PCS = df[df['fuel_type'] == fuel]['PCS_MJ/KG'].item() * ureg.megajoule / ureg.kilogram
lambda_air = 1
fuel_wet = get_fuel(fuel)
fuel_dry = delete_ash_water(fuel_wet)
elem_comp = elemental_composition(fuel_dry, elemental_comp=True)
reactivos, productos = get_reactants_and_products(elem_comp, razon_N2O2)
fuel_mass, air_mass, products_mass = get_mass_molecular(reactivos, productos)
fmass_flow = sum(fuel_mass.values()) * ureg.kilogram / ureg.hour
rac_mass, gases_secos, gases_humedos = get_rac(air_mass, fuel_mass, products_mass)
epa_flows = epa_flow(fuel, fmass_flow, PCS, lambda_air)
analytic_flows = analytic_flow(products_mass, fuel_mass, fmass_flow, gases_secos, gases_humedos)
analysis = EPA_analysis(epa_flows, analytic_flows)

In [ ]:
print('Análisis con exceso de aire')
fuel = 'lignite_coal'
#razon_N2O2 = 79/21
lambda_air = 1.12 #segun el profe
PCS = df[df['fuel_type'] == fuel]['PCS_MJ/KG'].item() * ureg.megajoule / ureg.kilogram
#fmass_flow = 100 * ureg.kilogram / ureg.hour
#PCS = 32.07 * ureg.megajoule / ureg.kilogram
fuel_wet = get_fuel(fuel)
fuel_dry = delete_ash_water(fuel_wet)
elem_comp = elemental_composition(fuel_dry, elemental_comp=True)
reactivos, productos = get_reactants_and_products(elem_comp, razon_N2O2)
reactivos, productos = get_excess_air(reactivos, productos, lambda_air)
fuel_mass, air_mass, products_mass = get_mass_molecular(reactivos, productos)
fmass_flow = sum(fuel_mass.values()) * ureg.kilogram / ureg.hour
rac_mass, gases_secos, gases_humedos = get_rac(air_mass, fuel_mass, products_mass)
epa_flows = epa_flow(fuel, fmass_flow, PCS, lambda_air)
analytic_flows = analytic_flow(products_mass, fuel_mass, fmass_flow, gases_secos, gases_humedos)
EPA_flux = EPA_analysis(epa_flows, analytic_flows)

In [ ]:
print(analytic_flows[0].to('cubic_feet/hour'))
print(analytic_flows[1].to('cubic_feet/hour'))
print(analytic_flows[2].to('cubic_feet/hour'))

In [ ]:
print(epa_flows[0].to('cubic_feet/hour'))
print(epa_flows[1].to('cubic_feet/hour'))
print(epa_flows[2].to('cubic_feet/hour'))

In [ ]:
fuel = 'bituminous_coal'
mcomb = 100 * ureg.kilogram / ureg.hour
PCS = 98.41 * ureg.megajoule / ureg.kilogram
epa_flows = epa_flow(fuel, mcomb, PCS)

In [ ]:
PCS = 98.41 * ureg.megajoule * 947.817 * ureg.british_thermal_unit / ureg.megajoule
PCS_2 = 1 / ureg.kilogram
fd = 9780 * ureg.cubic_feet
fd_2 = 1 / ureg.british_thermal_unit
mcomb = 100 * ureg.kilogram / ureg.hour
corrector = 0.0373
PCS*fd*mcomb*PCS_2*fd_2 / 10**6

In [ ]:
# Cálculo estequeométrico
# Cálculo con lambda nuevo (1.1)
# a * combustible + lambda * a (aire) = productos  a podría ser 2.97
x = 766.718/295.82 #kgCO2 / kg comb
xx = x / 44 #kmolCO2/kg comb
z = xx * 99.51 #kgcom/kmol comb
z # mol de CO2 para un mol de combustible

#140°C de temperatura de salida de carbono

In [ ]:
power_plant = 500 #MW
df = pd.read_csv('coal_composition.csv')
PCS = df[df['fuel_type'] == fuel]['PCS_MJ/KG'].item() * ureg.megaJ / ureg.kilogram
total_fuel_mass = round(sum(fuel_mass.values()),2) * ureg.kilogram
power_fuel = PCS * total_fuel_mass
power_fuel

In [ ]:
epa_flows = epa_flow(fname, fmass_flow)
analytic_flows = analytic_flow(products_mass, fuel_mass, fmass_flow)
EPA_analysis(epa_flows, analytic_flows)

In [ ]:
pm_aire = (cm.mw('O2') * aire_O2 + cm.mw('N2') * aire_N2) / 100
elem_comp

In [19]:
elem = {'C': 7.028,
 'H': 5.419,
 'O': 0.45,
 'Cl': 0.002,
 'S': 0.024,
 'N': 0.12,}
chemicals.combustion_data(elem)

CombustionData(stoichiometry={'CO2': 7.028, 'O2': -8.181249999999999, 'HCl': 0.002, 'SO2': 0.024, 'N2': 0.06, 'H2O': 2.7085}, HHV=-35118.71621588, Hf=3511681.11428412, MW=99.59422645999997)

In [ ]:
#Comparación con la EPA
fname = 'natural_gas'
fmass_flow = 1
def epa_flow(fuel_type, fuel_mass_flow_kghour, PCS_MJkg):
    df = pd.read_csv('coal_composition.csv')
    if fname in (df['fuel_type'].tolist()):
        fd = df[df['fuel_type'] == fuel_type]['fd'].item() * ureg.cubic_feet / ureg.british_thermal_unit
        fw = df[df['fuel_type'] == fuel_type]['fw'].item() * ureg.cubic_feet / ureg.british_thermal_unit
        fc = df[df['fuel_type'] == fuel_type]['fc'].item() * ureg.cubic_feet / ureg.british_thermal_unit
    else:
        corrector = (0.947817 / 2.2046216) * 1000
        #Dry list
        fd_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O']]
        fd_factors = [3.64,1.53,0.57,0.14,-0.46]
        fd = [a * b for a,b in zip(fd_list,fd_factors)]
        fd = sum(fd) * 10**6 / (PCS.magnitude * corrector) * ureg.cubic_feet / ureg.british_thermal_unit

        #Wet list
        fw_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O'], fuel_wet['H2O']]
        fw_factors = [5.56,1.53,0.575,0.14,-0.46,0.21]
        fw = [a * b for a,b in zip(fw_list,fw_factors)]
        fw = sum(fw) * 10**6 / (PCS.magnitude * corrector) * ureg.cubic_feet / ureg.british_thermal_unit

        fc = fw
    print(fd)
    PCS = PCS_MJkg * 947.817 * ureg.british_thermal_unit / ureg.megajoule
    print(PCS)
    fuel_mass_wu = fuel_mass_flow_kghour
    print(fuel_mass_wu)
    gas_flow_dry = fuel_mass_wu * PCS * fd / 10**6
    gas_flow_wet = fuel_mass_wu * PCS * fw / 10**6
    gas_flow_co2 = fuel_mass_wu * PCS * fc / 10**6
    return gas_flow_dry, gas_flow_wet, gas_flow_co2

#epa_flows = epa_flow(fname, fmass_flow)
#excess_air = 0.2

In [ ]:
gases_secos

In [ ]:
gases_secos* 2 * (ureg.meter)**3 / ureg.kilogram * fmass_flow

In [ ]:
def analytic_flow(products_mass, fuel_mass, fuel_mass_flow, gases_secos_flow, gases_humedos_flow):
    peso_molecular_gases = (cm.mw('O2') * aire_O2 + cm.mw('N2') * aire_N2) / 100
    P_est = scipy.constants.physical_constants['standard atmosphere'][0]
    rho_gas = cm.gas_density.rhog(peso_molecular_gases, P_est, 273.15)
    co2_mass_analytic = products_mass['CO2']
    co2_density = cm.gas_density.rhog(cm.mw('CO2'), P_est, 273.15)
    fuel_mass_analytic = sum(fuel_mass.values())
    mass_gas_co2 = (co2_mass_analytic / fuel_mass_analytic)
    #print(gases_secos_flow)
    #print(fuel_mass_flow)

    vol_gases_dry = gases_secos_flow / rho_gas * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    vol_gases_wet = gases_humedos_flow / rho_gas * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    vol_gases_co2 = mass_gas_co2 / co2_density * (ureg.meter)**3 / ureg.kilogram * fuel_mass_flow
    print(vol_gases_co2)
    print(vol_gases_dry)
    return vol_gases_dry, vol_gases_wet, vol_gases_co2

In [ ]:
def EPA_analysis(epa, analytic):
    EPA_dry, EPA_wet, EPA_CO2 = [abs(epa[i]- analytic[i]) / epa[i]*100 for i in range(len(epa))]
    return EPA_dry, EPA_wet, EPA_CO2
EPA_analysis(epa_flows, analytic_flows)

In [ ]:
fname = 'natural_sgas'
fuel_wet = {'C':75, 'H':5, 'O':10, 'Cl':0, 'S':1.5, 'N':1.4, 'H2O':0, 'Ash':7.1}
fuel_dry = delete_ash_water(fuel_wet)
mcomb = 1192.5 * ureg.kg / ureg.hour #masa_comb
generacion_vapor = 11116.48 * ureg.kg / ureg.hour
PCS = 27.54 * ureg.megaJ / ureg.kg
df = pd.read_csv('coal_composition.csv')
#PCS = 22200 * ureg.british_thermal_unit / ureg.lb
t_gases = (126.33 + 273.15) * ureg.degree_Kelvin
a_chimenea = 0.679 * ureg.m**2
SO2_medido = 0.03 #%
O2_medido = 8.3 #%
CO_medido = 89.44 #%
CO2_medido = 10.75 #%
hum_gases = 5.46 #%
p_atm = 94.925 * ureg.kpascal
p_est = 101.352 * ureg.kpascal
vel_gases = 9.75 * ureg.m / ureg.second
t_ambiente = (25 + 273.15) * ureg.degree_Kelvin
t_estandar = (20 + 273.15) * ureg.degree_Kelvin

In [ ]:
fuel_dry = delete_ash_water(fuel_wet)
elem_comp = elemental_composition(fuel_dry, elemental_comp=True)
reactivos, productos = get_reactants_and_products(elem_comp, razon_N2O2)
fuel_mass, air_mass, products_mass = get_mass_molecular(reactivos, productos)
rac_mass, gases_secos, gases_humedos = get_rac(air_mass, fuel_mass, products_mass)
fmass_flow =
analytic_flows = analytic_flow(products_mass, fuel_mass, fmass_flow)

In [ ]:
gases_humedos*mcomb

In [ ]:
megaj_btu = 947.817 * ureg.british_thermal_unit / ureg.megajoule
f_gases_est = mcomb * PCS * fw / 10**6 * megaj_btu

In [ ]:
pm_aire = (cm.mw('O2') * aire_O2 + cm.mw('N2') * aire_N2) / 100
p_est_pascal = p_est.to('pascal').magnitude
t_est_kelvin = t_estandar.to('degree_Kelvin').magnitude
p_amb_pascal = p_atm.to('pascal').magnitude
t_atm_kelvin = t_gases.to('degree_Kelvin').magnitude
rho_estandar = cm.gas_density.rhog(pm_aire, p_est_pascal, t_est_kelvin) * ureg.kilogram / ureg.meter**3
rho_ambiente = cm.gas_density.rhog(pm_aire, p_amb_pascal, t_atm_kelvin) * ureg.kilogram / ureg.meter**3
print(rho_estandar)
print(rho_ambiente)

In [ ]:
f_gases_actual = f_gases_est * rho_estandar / rho_ambiente
print(f_gases_actual.to('meter ** 3 / hour'))

In [ ]:
fw2 = 12634.764 * ureg.british_thermal_unit / ureg.pound
print(fw/fw2)

In [ ]:
vol_molar_seng = 1545.44 * 528 / 2116 #Ru*T/Pest  #T = Rankine #Ru = lbfpie / lbmol R°  #Pest = pi3/lbmol #Todo en medidas inglesas
vol_molar_si = R_molar * 293.15 / (P_est/1000)

In [20]:
#Equilibrar ecuaciones
#4 casos
#aCombustion completa perfecta
exceso_aire = 1.3
b = peso_molecular['C']
d = peso_molecular['H'] / 2
e = peso_molecular['S']
a = (2 * b + d + 2 * e) / 2
c = a * razon_N2O2 * exceso_aire
comb_perfecta = ['Aire', 'CO2', 'N2', 'H2O', 'SO2']
result = [a,b,c,d,e]
resultado_equilibrio = dict(zip(comb_perfecta, result))
print(resultado_equilibrio)
#Combusción completa con exceso de aire
#Combustión imcompleta deficit de aire
#Combustión incompleta bajo exceso de aire

NameError: name 'peso_molecular' is not defined

In [ ]:
reactantes = ['C', 'H', 'S', 'O', 'N']
dict_reactantes = {}
dict_reactantes['O'] = 2 * resultado_equilibrio['Aire']
dict_reactantes['N'] = 2 * 3.76 * resultado_equilibrio['Aire']
for element in reactantes[:3]:
    if element != 'O' or element != 'N':
        dict_reactantes[element] = peso_molecular[element]
masa_reactantes = {}
for element in dict_reactantes:
    masa_reactantes[element] = dict_reactantes[element] * cm.mw(element)

In [ ]:
producto = comb_perfecta[1:]
masa_productos = {}
for element in producto:
    masa_productos[element] = resultado_equilibrio[element] * cm.mw(element)

In [18]:
import thermopy
#Cálculo del punto de rocío de los gases
presion = 101.352 #kPa
N_molar = resultado_equilibrio.copy()
n_H2O = N_molar['H2O']
del N_molar['Aire']
n_total = sum(N_molar.values())
presion_parcial_H2O = presion * n_H2O / n_total
sat_steam=IAPWS97(P=presion_parcial_H2O/1000,x=1)
t_punto_rocio = (sat_steam.T - 273)
print('La temperatura de punto de rocío es la siguiente: ', round(t_punto_rocio,2), '°C')

NameError: name 'resultado_equilibrio' is not defined

In [ ]:
from thermo.chemical import Chemical
tol = Chemical('coal')

In [21]:
from chempy import balance_stoichiometry
reactivos, productos = balance_stoichiometry({'CH4', 'O2'},{'CO','CO2', 'H2O'})
print(reactivos)
print(productos)

OrderedDict([('CH4', 2*x1), ('O2', 3*x1 + 2)])
OrderedDict([('CO', 2*x1 - 4), ('CO2', 4), ('H2O', 4*x1)])


In [ ]:
'''
Hull dimensions and load lines
This chapter starts by giving the defini- tions of the expressions used in various situations for length, draught and breadth, followed by the definitions of load lines, which describe how much of the hull that is submerged.
Lengths
The overall length of the ship LOA is nor- mally of no consequence when calcu- lating the hull’s water resistance. The determining factors used are the length of the waterline LWL and the so-called length between perpendiculars LPP. The dimensions referred to are shown in Fig. 1.01.
The length between perpendiculars is the length between the foremost per- pendicular, i.e. usually a vertical line through the stem’s intersection with the waterline, and the aftmost perpendicu- lar which, normally, coincides with the rudder stock. Generally, this length is slightly shorter than the waterline length, and it can often be expressed as
Draught
The ship’s draught, typically denoted T in literature, is defined as the vertical distance from the waterline to the point of the hull which is deepest in the wa- ter, see Figs. 1.01 and 1.02. The fore- most draught TF and aftmost draught TA are normally the same when the ship is in the loaded condition, i.e. no trim.
The “scantling draught” is the distance from the keel to the summer load line, see the section “Load lines”.
Ballast draught is the draught of the ship with no cargo but adequate ballast water to ensure the stability of the ship.
Generally, the most frequently occur- ring draught between the scantling and the ballast draught is used as the “de- sign draught”.
Breadth
Another important factor is the hull’s largest breadth on the waterline BWL, see Fig. 1.02.
Depth
The depth describes the distance from the keel to the underside of a given deck of the ship and is typically denot- ed D. The moulded depth describes the distance from the keel to the freeboard deck measured at the ship side.
Load lines
Painted halfway along the ship’s side is the load line mark (also known as the Plimsoll Mark after its inventor), see Fig. 1.03. The lines and letters of the load line mark, which conform to the free- board rules laid down by the IMO and local authorities, indicates the draught to which the ship may be loaded. The term freeboard refers to the minimum distance from the lowest watertight deck of the ship to the waterline, i.e. when fully loaded, the load line.
There are, e.g. load lines for sailing in freshwater and seawater, respectively, accounting for the difference in the mass density of water and the corre- sponding difference in buoyancy de- pending on the salinity. Further divi- sions for tropical conditions and summer and winter sailing are given. According to the international free- board rules, the summer freeboard draught for seawater is equal to the “scantling draught”.
The winter freeboard draught is less than that valid for summer because of the risk of bad weather. Correspond- ingly, a lower freeboard can be allowed in tropical waters. A further description of a ship’s stability is outside the scope of this paper.
Size determining factors
Displacement, deadweight and lightweight
When a ship in loaded condition floats at an arbitrary water line, its displace- ment is equal to the relevant mass of water displaced by the ship. Displace- ment is thus equal to the total weight, of the relevant loaded ship, normally in seawater with a mass density of 1.025 t/m3.
Displacement comprises the ship’s lightweight and its deadweight, where the deadweight is equal to the ship’s loading capacity, including bunkers and other supplies necessary for the ship’s propulsion. The deadweight at any time thus represents the difference between the actual displacement and the ship’s lightweight, all given in tons.
The word “ton” does not always ex- press the same amount of weight. Be- sides the metric ton (1,000 kg), there is the English ton (1,016 kg), which is also called the “long ton”. A “short ton” is approx. 907 kg.
The deadweight tonnage (dwt) is based on the ship’s loading capacity including the fuel and lube oils, etc. required for the operation of the ship. It is often used as an indication of size, referring to the loading capacity when the ship is loaded to its scantling draught, i.e. to the summer load line.
Sometimes, the deadweight tonnage may also refer to the ship’s loading ca- pacity at the design draught, if so, this will be mentioned.
The lightweight of a ship describes the weight of the ship itself in light condi- tion, i.e. without load on the ship. The lightweight tonnage (lwt) is normally not used to indicate the size of a ship.
Table 1.01 indicates the rule-of-thumb relationship between the ship’s, dead- weight tonnage (summer freeboard/ scantling draught) and lightweight ton- nage.
The designation “payload” describes the weight that can be loaded onto the ship, excluding bunkers and supplies necessary for the ship’s propulsion.
Gross and net tonnage
Without going into detail, it should be mentioned that there are also such measurements as the dimensionless gross tonnage (gt) and net tonnage (nt), which are calculated on the basis of the ship’s volume and a multiplier defined by the IMO in the International Conven- tion on Tonnage Measurement of Ships. Gross tonnage describes the entire en- closed ship space, whereas net ton- nage covers only the cargo spaces. These measurements often form the basis for different regulatory require- ments from the IMO.
Gross and net tonnage should not be confused with the older pre-convention terms gross register tons (grt), and net register tons (nrt) where 1 register ton corresponds to 100 English cubic feet, or 2.83 m3.
Coefficients related to the hull
The measures of length, breadth, draught and depth as well of weights can with some additional measures be used for calculating some dimension- less coefficients.
Block coefficient
Various form coefficients are used to express the shape of the hull. The most important of these coefficients is the block coefficient CB, which is defined as the ratio between the displacement volume ∇ and the volume of a box with dimensions LWL x BWL x T, see Fig. 1.02.
In the case cited above, the block coef- ficient refers to the length on waterline LWL. However, shipbuilders often use block coefficient CB,PP based on the length between perpendiculars, LPP, in which case the block coefficient will, as a rule, be slightly larger because LPP is normally slightly shorter than LWL.
Table 1.01 shows typical block coeffi- cients for different categories of ships, indicating a strong correlation between ship speed and block coefficient.
Water plane area coefficient
The water plane area coefficient CWL expresses the ratio between the ships’s waterline area AWL and the product of the length LWL and the breadth BWL of the ship on the waterline, see Fig. 1.02.
Generally, the water plane area coeffi- cient is some 0.10 higher than the block coefficient.
CWL ≅ CB + 0.1
This difference will be slightly larger on fast ships with small block coefficients where the stern is also partly immersed in the water and thus becomes part of the ”water plane” area.
deadweight = displacement – lightweight
Midship section coefficient
A further description of the hull form is provided by the midship section coeffi- cient CM, which expresses the ratio be- tween the immersed midship section area AM (midway between the foremost and the aftmost perpendiculars) and the product of the ship’s breadth BWL and draught T, see Fig 1.01.
In general, the midship section coeffi- cient for merchant (displacement) ships is high, in the order of 0.95-0.99. Low values are found for fast ships such as container ships, whereas high values are found for the slower bulkers and tankers. Fast ferries and naval ships can have significantly lower values.
Prismatic coefficient
The longitudinal prismatic coefficient CP expresses the ratio between dis- placement volume ∇ and the product of the midship frame section area AM and the length of the waterline LWL, see also Fig. 1.02.
The prismatic coefficient describes how voluminous the fore and aft parts of the ship are. A low prismatic coeffi- cient indicates a voluminous midship section compared to the fore and aft parts.
Fineness ratio alias length/displace- ment ratio
The length/displacement ratio or fine- ness ratio, CLD, is defined as the ratio between the ship’s waterline length LWL, and the length of a cube with a volume equal to the displacement volume.
Longitudinal centre of buoyancy
LCB expresses the position of the cen- tre of buoyancy and is defined as the distance between the centre of buoy- ancy and the midpoint between the ship’s foremost and aftmost perpendic- ulars, termed “midships”. The distance is normally stated as a percentage of the length between the perpendiculars.
Whether LCB is positive fore of the midpoint between perpendiculars, mid- ships, or abaft midships is not strin- gent. The Holtrop Mennen, Ref. [1.3], power prediction method defines LCB positive fore of midships, whereas the method of Ref. [1.2] defines LCB posi- tive abaft midships.
Relations between LCB and speed can be formulated for LCB. When measured positive fore midships: For a ship de- signed for high speeds, e.g. container ships, the LCB will, normally, be nega- tive, whereas for slow-speed ships, such as tankers and bulk carriers, it will normally be positive. The LCB is gener- ally between −3% and +3%.
Longitudinal centre of flotation
LCF is the geometrical centre of the waterplane area, which does not nec- essarily coincide with the midship posi- tion, due to the ship usually not being symmetrical around this point in the longitudinal direction. The ship trims about this point and not LCB.
 CM = A M BWL × T
  CP=∇= ∇ =CB,WL AM × Lwl CM × BWL × T × LWL CM
CLD =
LWL

Ship types with engine applications in overview
Depending on the nature of their cargo, and also the way the cargo is handled, ships can be divided into different cate- gories, some of which are mentioned in Table 1.01 along with typical number of propeller(s) as well as engine type.
The three largest categories of ships are container ships, bulk carriers (for bulk goods such as grain, coal, ores, etc.) and tankers, which again can be divided into more precisely defined sub-classes and types.
Furthermore, Table 1.01 includes gen- eral cargo ships, where different kinds of cargo is brought onboard in pieces, and roll-on/roll-off (ro-ro) ships carrying lorry trailers and other cargoes that can be rolled on board. Rolling cargo in the form of cars and lorries are also com- bined with passenger transport in ro- pax ships.
Table 1.01 provides only a rough out- line. In reality, there are many sub-types of ships, as well as combinations of these sub-types, i.e. ships carrying ro- ro cargo in the hull and containers stacked on deck.
As seen in Table 1.01, two-stroke en- gines are broadly used as main propul- sion engines. Two-stroke diesel cycle engines have the highest efficiencies amongst mechanical means of propul- sion, see Chapter 3.
This is reflected in the fact that two- stroke engines practically are the only engine of choice for ship types where fuel prices represent the main running costs, for example tankers, bulk carri- ers and container ships engaged in lin- er traffic between the continents.
For specialised ships, such as ferries and cruise ships, other parameters can constitute the main economic factors. The higher power density and lower height of four-stroke engines can make them more attractive in such cases.
For large ocean-going ships, fixed pitch propellers (FPP) are the most common choice, whereas controllable pitch pro- pellers (CPP) are often utilised for smaller ships calling on smaller ports which demand increased manoeu- vrability. In the case of a two-stroke main engine, it is possible to couple the propeller directly to the main engine, as an engine with an rpm matching the propeller optimum can be found. For
four-stroke engines, the higher rpm of the engine requires a reduction gear- box. Propeller and engine matching is elaborated in Chapter 3.
Efficiencies affecting the total fuel consumption
The total fuel power, Pfuel (power deliv- ered through the fuel), required for pro- pelling a ship through water is gov- erned by the fuel equation. This section describes the parameters included in the equation, and references are made to other parts of this paper where these are explained in detail.
The resistance of the hull, RT, is influ- enced by multiple parameters as de- scribed in the section “Resistance and influencing parameters”. In general, the resistance is proportional to the speed to the power of i = 2 to 3, R ∝ k × Vi. This means that when increasing the speed with the power of 1 (linearly), the power required will increase with the speed to a higher power, P ∝ k × Vi+1.
These relations are further discussed in Chapter 2, section “Propeller law and power/speed curves”.
The design speed of the ship is typical- ly set according to the desire of the owner, if not limited due to regulations. In the fuel equation, the product of the resistance and speed is divided by the efficiencies of all of the components in- volved in propelling the ship - remem- bering all the efficiencies to consider is easy, as the indices combined spells HORSE.
ηH is the hull efficiency. It takes into ac- count the difference in the effective (towing) power and the thrust power that the propeller must deliver to the water because the hull accelerates the water and the propeller sucks water past the hull. This is further elaborated in Chapter 2, section “Flow conditions”.
ηO is the open water efficiency of the propeller (not behind the hull), giving the efficiency of the propeller working in a homogeneous wake field with flow perpendicular to the propeller and is further investigated in Chapter 2, sec- tion “Efficiencies and influencing pa- rameters”.
ηR is the relative rotative efficiency of the propeller, accounting for the chang- es in water flow to the propeller behind the ship relative to the open water flow conditions, as further described in Chapter 2, section “Rotative efficiency”.
ηs accounts for the efficiency of the shaft connecting the main engine and propeller. It depends on the length of the shaft, whether gearing is necessary, etc. This is further described in Chapter 2, section “Shaft efficiency”.
ηE describes the efficiency of the en- gine, depending on type of design and a variety of other parameters described in Chapter 3.
Resistance and influencing parame- ters
The magnitude of resistance on the ship’s hull, RT, is naturally paramount for the power required to move the ship (as seen in the fuel power equation) and hereby the fuel consumption.
The following section will explain the physical phenomena giving rise to the different components of the total calm water resistance, also termed the source resistances, and quantify their contribution to the total resistance.
Components of resistance
A ship’s calm water resistance is partic- ularly influenced by its speed, displace- ment, and hull form. The total resis- tance RT consists of many source-resistances R, which can be di- vided into three main groups, viz.:
1) Frictional resistance, RF 2) Residual resistance, RR 3) Air resistance, RA
The influence of frictional resistance depends on the wetted surface of the hull, whereas the magnitude of residual resistance describes the energy lost by the ship setting up waves, eddies and by the viscous pressure resistance, which all depend on the hull lines. For slow moving ships such as tankers and bulkers, the frictional resistance is often of the greatest influence (70-90%) whereas for fast going ships, such as panamax container carriers, the fric- tional resistance may account for as lit- tle as half of the combined resistance.
Air resistance normally represents about 2% of the total resistance, how- ever, with a significant increase up to approx. 10% for ships with large super- structures such as container ships with containers stacked on deck. If wind re- sistance is considered, the figures may increase.
Water with a speed of V and a density of ρ has a dynamic pressure of:
pdyn= 1⁄2 × ρ × V2 “Bernoulli’s law”
Thus, if water is being completely stopped by a body, the water will react on the surface of the body with the dy- namic pressure, resulting in a dynamic force on the body.
This relationship is used as a basis when calculating or measuring the source-resistances R of a ship’s hull, by means of dimensionless resistance co- efficients C. Thus, C is related to the reference force K, defined as the force that the dynamic pressure of water with the ship’s speed V exerts on a surface which is equal to the hull’s wetted area AS. The rudder surface is also included in the wetted area. The general data for resistance calculations are thus based on the following two important rela- tions:
Reference force: K = 1⁄2 × ρ × V2 × AS
Source resistances: R = C × K
On the basis of many experimental tank tests, and with the help of pertaining di- mensionless hull parameters, methods have been established for calculating all the necessary resistance coeffi- cients C and, thus, the pertaining source-resistances R at an early proj- ect stage without testing, Ref. [1.2] and [1.3].
In practice, the approximate calculation of a particular ship’s resistance, which is required for the initial dimensioning, is often verified and optimised by test- ing a model of the ship in a towing tank.
Frictional resistance, RF
The frictional resistance RF accounts for two effects: Firstly the friction of a flat plate of the same length as the hull and an area equivalent to the hull’s wet- ted surface, AS. Secondly the frictional resistance caused by the curvature of the ship’s hull compared to a flat plate, see Fig. 1.05. The magnitude of the fric- tional resistance increases with the fouling of the hull, i.e. by the growth of algae, sea grass and barnacles.
RF = CF × K
The wetted surface area As can be esti- mated based on Mumford’s formula. As a rule of thumb, it can be estimated by this formula within 15% accuracy.
Residual resistance, RR
Residual resistance RR comprises wave making resistance and viscous pres- sure resistance. Sometimes the addi- tional frictional resistance due to the curvature of the ship’s hull are also in- cluded in the residual resistance.
Wave making resistance refers to the energy lost by the setting up waves during the ship’s propulsion through water, and will typically, depending on the ship speed, form the greater part of the residual resistance, see Fig. 1.05.
The viscous pressure resistance is usu- ally a small part of the total resistance. It arises from separation losses in the boundary layer and the increased thick- ness hereof along the ship’s side. The result of increased boundary layer thickness is a lower pressure on the aft of the ship than on the fore part, which gives rise to resistance. The concept of the boundary layer is explained in Chapter 2, section “Flow conditions”.
Generation of eddies when the flow separates from the hull surface is a special part of the viscous pressure re- sistance, which results in separation
losses, sometimes referred to as eddy resistance. Eddies are created both due to separation in the boundary layer and on the aft part of the ship, due to the abrupt changes in curvature found here.
The procedure for calculating the spe- cific residual resistance coefficient CR is described in specialised literature, Ref. [1.2].
RR = CR × K
The residual resistance is typically very limited at low ship speeds, see the sec- tion “The Froude number”, but it can be significant at elevated ship speeds.
This results in the combined resistance growing by as much as the cubic of the speed or even more. Accordingly, the required power and, hence, fuel con- sumption can be proportional to the speed to the power of four, P ∝ k × V4 or even more, for fast ships. See also “Propeller law and speed power curves” in Chapter 2.
Air resistance, RA
In calm weather, air resistance is, in principle, proportional to the square of the ship’s speed, and the cross-sec- tional area of the ship above the water- line. It is important to distinguish air re- sistance from wind resistance, air resistance only accounts for the resis- tance from moving the ship through the air with no wind.
The air resistance can, similar to the foregoing resistances, be expressed as RA = CA × KA. Using a drag coefficient CA and an area of the ship above the water line Aair, the resistance can be ex- pressed as:
RA = 1⁄2 × ρair × CA × Aair × V2
Typically, the drag coefficient can be estimated at approx. CA ≈ 0.9 depend- ing on the shape of the superstructure, whereas the area above the waterline, on which the wind acts, must be calcu- lated because the area depends on the wind direction. Further investigations on air resistance have been done in Ref. [1.4].
Parameters influencing resistance and optimisation hereof
Numerous parameters influence the to- tal calm water resistance of a ship, and the nature of the influence can differ significantly depending on the speed of the ship. The following section provides an overview of the influencing parame- ters and a brief introduction to their re- lation. For in-depth understanding the reader is advised to consult special lit- erature such as Ref. [1.2].
The Froude number
The Froude number is a dimensionless coefficient especially important for the resistance of ships. Ships with a similar Froude number will experience similar relative amounts of source resistance (relation between CF, CR, CA,) despite not being the same size and moving at different speeds.
The total resistance, and thus fuel con- sumption, increases when the Froude number increases. This is because the wave making resistance increases sig- nificantly when the Froude numbers ex-
ceed approx. Fn = 0.16 to 0.17, as can be seen in Fig. 1.05 from Ref. [1.2]. From this figure it is also clear that the friction is practically the only influenc- ing parameter at Fn < 0.15.
In order to exemplify the influences of the different terms in the Froude num- ber, a parameter sweep is displayed in Table 1.02.
When considering the formula for the Froude number, it becomes evident that by increasing the length of the ship, the value of the Froude number will decrease. Of course, this does not come without a price, as it increases the wetted surface area of the ship, and hereby the frictional resistance. Depending on the speed of the ship, this increase in absolute frictional resis- tance may in some cases be meaning- ful.
Typically, the length of the ship is the most costly parameter to adjust with regards to capital costs, but consider- ing the EEDI regulations as described in Chapter 4 an increased length can be necessary if a specific speed is re- quired for a special trade.
On the other hand, fuller container ships with larger block coefficients are seen now, as the speed of such ships has dropped significantly.
Influence of wetted surface area
The wetted surface area is paramount for the frictional resistance of the ship, which is seen to be predominant at
Fn < 0.15 in Fig. 1.05.
By increasing the block coefficient, the wetted surface area is reduced relative to the cargo amount that can be car- ried, which is why slow-going ships such as tankers and bulkers have a high block coefficient.
The wetted surface area can be esti- mated by Mumford’s formula, see sec- tion “Frictional resistance, RF“.
Influence of block coefficient
The block coefficient describes the overall fullness of the hull. It is a good and easy-to-calculate indicator of a hull’s resistance and capabilities with regard to speed. However, the other parameters described in this section may have a relatively larger influence, specifically on the resistance, than small changes to the block coefficient. In general, fast ships, for which the wave making is the primary resistance parameter, demand small block coeffi- cients, whereas slower ships, for which the friction against the hull is the main resistance parameter, are most effec- tive with a higher block coefficient.
Influence of the fineness ratio alias length/displacement ratio
The fineness ratio describes the length of the ship relative to the displacement, and influences the resistance, especial- ly for faster ships. When increasing the fineness ratio for constant displace- ment the residuary resistance decreas- es. However, at the same time the wet- ted surface area is increased. The optimum length of the specific hull, with its specific design speed, must be found with due consideration to the lim- itations imposed by canal passages, harbours, production costs etc.
Influence of the prismatic coefficient
The prismatic coefficient is an import- ant measure for describing how well the ship displaces the water while mov- ing through it. The optimum for the prismatic coefficient changes greatly with the Froude number of the ship, and the reader is advised to consult specialised literature.
Added resistance in various conditions
Shallow water
In general, shallow water will have no influence when the water depth is more than 10 times the ship draught. Shallow water may increase the resistance of the ship significantly for a variety of rather complex reasons listed below. The reader is advised to consult special literature e.g. Ref. [1.5].
The pressure set up by the ship’s mo- tion through the water will be greater as the flow of the water is restricted, there- by retarding and increasing the size of the wave system set up by the ship, and resulting in added resistance. This effect is primarily for Fn > 0.2.
In addition, water will flow back around the ship due to the restriction of the waterway caused by the presence of the ship (similar to an increase in veloc- ity due to a local obstruction in a pipe). Furthermore, the propeller will to some extent suck water faster past the ship surface than in deep water. Both these effects increase flow velocities and, therefore, also the resistance.
In addition, the so-called squat effect may occur. The squat effect means that the pressure underneath the ship is re- duced as the limited gap between the seabed and the ship results in friction on the water under the hull. The result is that water will flow to the side instead of underneath the ship. The squat ef- fect results in a larger than normal draught and hereby increased resis- tance. This is also primarily seen at ele- vated Froude numbers.
Weather, high seas
Waves created by high winds and swells can have a significant effect on the resistance experienced by the ship. Especially if the ship length and wave length are about equal.
The waves will set the ship in motion and lead to added resistance as more water is affected by the movement of the ship. Additionally the waves from the seaway will reflect on the hull, and thereby increase the resistance as well.
In combination, this is termed the wave drift force. The influence hereof can be seen if an object is left to float freely in the waves. Such an object would slowly move in the direction of the wave prop- agation.
Furthermore, rudder corrections will be much more frequently needed to stay on course. The use of the rudder will of course result in increased resistance, this effect being most predominant for small ships (LPP < 135 m ≈ 20,000 dwt) due to their lower directional stability. Additionally, the shape of the stem can greatly influence the ship’s ability to “cut through” high seas.
Weather, wind
It is important to distinguish between air and wind resistance. Air resistance only accounts for the calm weather re- sistance from moving the ship through the atmosphere without any wind. Wind resistance is calculated in the same way as air resistance but takes into ac- count the combined speed of the ship and the wind.
For large ocean-going ships, wind re- sistance will normally be significantly lower than the wave resistance, but for ships intended for trade in sheltered waters without large wave resistance, e.g. ferries, the wind resistance can be the most significant added resistance in heavy weather.
Fouling
Fouling can result in a substantial in- crease in frictional resistance. Extreme cases have been seen where fouling had increased the frictional resistance by as much as 100%, normally up to 20-30%.
Before IMO banned TBT (tributyl tin) for new applications from 2003, and the full ban from 2008, marine growth on the hull was slower, but TBT was banned as it is extremely toxic to the marine environment. This means that the management of the ship with re- gard to dry-docking, hull cleaning and propeller polishing must be optimised, and that the cost of mechanical clean- ing measures and the cost of added fuel consumption due to fouling must be balanced.
When designing the ship, especially for trade in warmer seas where growth is fast, some margin must be included for fouling.
Similar to the fouling, the increased roughness of the hull over time, arising from dents from interactions with the quay, etc., will increase the resistance of the hull.
Ice
Ships sailing in ice will experience sig- nificantly higher frictional resistance. Designing for icy conditions is a spe- cialist area. The reader is advised to consult the separate paper “Ice Classed Ships, Main Engines” and spe- cial literature on resistance in ice. An rpm extended engine load diagram can be of relevance to ice classed ships, see “Rpm extended load diagram” in Chapter 3.
Resistance margins in a slow steaming environment
To account for average weather , a rela- tive resistance margin (called a sea margin) has traditionally been added to the power requirement for propelling the ship in calm waters at the design speed, as given, for example, in Ref. [1.2]. Lately the ship design speeds have been lowered as a result of the in- creased focus on fuel consumption and environmental impact, see Chapter 4. This has led to a lower power require- ment and, therefore, smaller engines. If the traditional relative resistance mar- gins are applied to these smaller en- gines, the resulting absolute power margin will also be smaller.
Nevertheless, the weather does not care whether a ship of the same size is designed for a reduced service speed or not. The absolute added resistance experienced on the hull will be the same. Therefore, it is important for the ship designer to consider resistance margins for the specific project. The designer must secure that the engine selected can ensure safe manoeuvring in all relevant conditions, and that all relevant operating points (power, rpm) for the propeller are inside the engine’s load diagram. See the engine selection spirals in Chapter 3 for further guidance on this.



'''

In [ ]:
'''
Hitos importantes

-1908 (Mayo), Wilbur W. es el primero en llevar a una persona.
-1908 (Septiembre), Primera persona al morir en un accidente aéreo.
-1909 (Julio), Bleriot cruza el canal de la mancha.
-1910 (Agosto) Copetta realiza el primer vuelo en Chile.
-1912 (Abril) Creación de la RAF.
-1914 Primeros combates aéreos. Alemania fabrica cerca de 1.000 aviones.
-1914 Primer vuelo comercial en Florida.
-1914 Primer derribo.
-1917 Vuela por primera vez en la guerra Manfred Von Richtofen.
-1924 Primera vuelta al mundo (USAAF).
-1930 Supremacía del monoplano. Boeing Monomail, 1930.
Tren retráctil, construcción de metal, carenado para el motor radial, alas cantiléver, 700kgs de carga útil
-2da Guerra Mundial
	-Mejoras en motores, hélices.
	-Introducción de flaps.
	-Supremacía del monoplano.
	-Aviónica, uso de radar.
	-Auge de los bombarderos.
-1961, Primer viaje al espacio, Yuri Gagarin
-1967, X-15 logra alcanzar Mach 6.1.
-1976, Concorde comienza a operar.
-1977, Mayor accidente de la aviación, Tenerife.
-1986, Explota el transbordador espacial Challenger.
-1988, Primer vuelo del AN-225.
	-Aviónica, uso de radar.
	-Auge de los bombarderos.
-2000s Incorporación de UAV de manera masiva en la aviación militar
-Ala.
-Superficies de mando.
Fuselaje.
Empenaje.
Cabina.
Tren.
Motores.

Sistema Hidráulico.
Sistema de Propulsión.
Sistema de controles de vuelo.
Sistema de Combustible.
 Sistema Eléctrico.
Sistema de Presurización.
Aviónica.


Introducción
   Breve historia del Transporte
                              1850 1875 1900 1925 1950 1975 2000 2025 Año
         J. Vergara
       Introducción
    Aplicaciones tecnológicos tempranas
         J. Vergara

       Introducción
    También el transporte de recursos y electricidad
          J. Vergara

       Introducción
    La energía es vital en el transporte marítimo
      2a clase, e
(aparte de aceleraciones y cambios de
ra
tamaño
-
altu
Con la teoría hidrodinámica desarrollada por W. Froude, es posible estimar la demanda unitaria
Como vimos en la
l consumo
estacio
   nario
-
     )
depende de la
, para vencer la repulsión EM
y del .
de un vehículo
velocidad
     -
forma
    Se puede conocer por
demanda
buque
qué y cómo
para navegar en el mar.
cierto
   energía
      de
del propulsor
potencia
efectiva
. Con la forma del buque y
se estima,
emisiones, y se agrega en el sector.
     consumos y
la
potencia al freno
, los
     J. Vergara

       Introducción
    Forma en la era de los galeones
      El
knarr
progresivamente el remo por velas, aspirando a mayores distancias.
Tres, Dos y As (3:2:1)
Para los Galeones se aplicaba 4:2:1, más lleno que las Galeras y más esbeltos que las Naves, cuyo puntal servía para proteger a la artillería.
diseño
de drakkar
los galeones
vikingos a las galeras. Dejaron
naves derivó de los
    y
     proporciones
de los buques del S.XIV eran
Las
empíricas, i.e. Fórmula Catalana (o Murciana):
      , o bien L= 3
·
B, B= 2
·
H.
    J. Vergara

       Introducción
    Primeros cascos basados en modelos
    Como vimos en la clase 1,
la forma del
casco fue .
   empírica, hasta
Este ingeniero, aparte de destacar en más de 100 puentes y túneles, desarrolló buques adoptando
Isambard Brunel
    los resultados experimentales de
William Froude
.
          Buque D
(
)L(
)
Propulsor
P(
)
V(
)
T
m
HP
kn
  Great Western
Great
Great Eastern (
(
Britain (
) 2.300 65 R ) 3.270 98 H
)
1838
ueda 400 9 élice 1.400 12
+
           1843
 1858
27.400 211 R
H
+Vela
2.600 14
                        J. Vergara

      Tipos de Buques
      Algunas definiciones básicas
del sector
       Barco
-
: Vehículo flotante de
, hierro u otro material que,
●
es útil para navegaciones o presas marítimas de importancia
ma
  dera
impulsado por viento, remos o diversos tipos de motores, para transportar personas o bienes.
Mercante
Costeros Oceánicos Soporte
Guerra
Superficie Submarino
Buque capaz de navegar bajo la superficie
             ●
Buque
: Barco con cubierta que, por su tamaño, solidez y fuerza,
● ● ●
       em
-
● ●
      (del latín
interior, casco de nave)”.
buc
, vientre, capacidad
     J. Vergara

      Tipos de Buques
      E
stado actual del sector marítimo
  Los buques han sido mejorados durante siglos de desarrollo y progreso, y son parte integral del
    sistema de transporte
de bienes y servicios, jun
   to a los buques de guerra, encargados de
- -
prote las líneas de comunicaciones marítimas.
  ger
Aparte de 2.3 millones de botes pesqueros en el
     mundo, hay casi
ton), con 1
Los buques de guerra, al margen de patrulleros,
35000 naves comerciales
(1000 , que transportaron 11
+
     billón de
tons
+
billones de toneladas de carga (7%/a) en el 2019.
  suman más de
1250 unidades
.
    J. Vergara

      Tipos de Buques
     Clases de buques mercantes por su alcance
     Costeros
Carga ● Graneleros, madereros, barcazas,...
Soporte
Remolcador Empujador Cableador Buque faro Rompehielos Perforador Dragas, etc.
Oceánicos
       ●
Carga ● Granel, petróleo, mineral, ●
containers, LNG, reefers,... Pasajeros ●
           Pasajeros y Autos ● Ferries, hovercrafts,
 ●
hydrofoil, placer,... ●
Transatlánticos ● Mixto
          Mixto
●
Pasajeros, autos, y carga. ●
Industriales Pesca, extracción minera,...
Pasajeros y carga ● Industriales ●
        ●
Pesca, factoría, oceanografía,... ●
        J. Vergara

      Tipos de Buques
     Clases de buques mercantes por su especialidad
     Transporte
Industrial
Pesqueros ● Factorías ●
Cableador ● Perforador ● Sumergible ● Incinerador ● Grúas, dragas ●
Invest
Servicio
Remolcador Empujador
Buque grúas Apoyo buceo Bomberos Pilotaje Alivianador Antiderrame
Rec
       ● ●
Pasajeros ●
Porta
Tanquero
Carga general Granelero
OBO (ore Madereros,
Tow Levante
        Ferries
RoRo ●
,
contenedor ●
       ●
-
       ●
●
●
gasero ●
-
      ●
               ●
●
bulk
oil
-
-
● )● -●
           Tug , barcas.
hidrográ
.(
-● -●
         ●
fico fico
oceanográ , ex. sismos)
,
.
Booster
        J. Vergara

      Tipos de Buques
     Clases de buques por tipo de propulsión
   Buques de Guerra
Buques Mercantes
Diesel Lento
Dominante en buques grandes
Diesel Media Velocidad Buques pequeños
Turbina a Vapor
Extinguida
    ● ● ● ● ● ●
Diesel Rápido Lanchas rápidas, Misileras
Diesel Media Velocidad Corbetas, Fragatas, auxiliares
●
●
●
               Turbina a Vapor
Caldera
Caldera Buques grandes, en extinción.
-
-
         -
Combinadas (Co..[And/Or]..) Buques medianos a grandes
-
Turbina a Vapor
Nuclear Submarinos, Portaaviones, Cruceros
Turbina a Vapor
●
●
●
Nuclear A futuro, de todos modos
      Turbina a Gas Buques de todo tipo
Turbina a Gas Aplicaciones crecientes
Vela
Asistencia propulsiva
           J. Vergara

      Tipos de Buques
      E
stado actual del sector marítimo
  El tonelaje comercial es aproximadamente:
  375 Mt 358 Mt 109 Mt 103 Mt
55 Mt
Tanqueros Graneleros Portacontenedores
general
       3.0 Mt 1.4 Mt 0.5 Mt 0.4 Mt 1.5 Mt
Estados Unidos Rusia
Gran Bretaña China
Resto del Mundo (?)
Carga Otros
   El tonelaje naval, la fracción por país es:
           J. Vergara

     Tipos de Buques
    Clases de vehículos por forma de sustentación
  Vehículos
     aeroplano aeróstato
aerodinámico
      aerostático
híbrido
hidrostático
sumergible
híbrido
hidrodinámico
hidroala
                 avión E.S. (SEA)
s
lizador (ACV)
buque aerode
monocasco
multicasco
semi
catamaran SWATH
casco de planeo
     E.S. (SES)
profundo convencional
total
rompeola
secante
sumergido
                                   J. Vergara

     Tipos de Buques
    Clases de vehículos por forma de sustentación
      Escala El Relativa estado
combatir
el
(
buque
debe del mar,
flotar navegar
        Costo Específico
Potencial de Carga
tolerando
Energía
)
          y
misión
,
, , etc.).
cumplir
(
su
transportar
      comunicar
   Potencial de Velocidad
Potencial de Comportamiento Marinero
                   aerodinámico
aerostático hidrostático hidrodinámico
      J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
   formas
potencia
         Velocidad
Potencia
Costo Específico
Comportamien to Marinero
Capacidad de Carga
= f (S W
    à
V
,P
)
,C
tamaño
D
s
diseño, materiales
   B
S
C.P.
         C
- SK
arreglo
)
formas
= f (Tecnología, V,
C.P.
               D
 =f(
, B, T, I WP
,C
,C
, P
)
,C
     G
C.P.
C.P.
              DWT
= f (L, D, B, P S
payload
,C D
B
)
        J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
      manga, B
A.P.
                       eslora línea de agua
L WL
eslora total Loa
L
eslora entre PP Lpp
arrufo
F.P.
L
W
puntal,
calado, T
                   D
  L
   J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
      D
D
DC
=s
= vol =
=
r 1.025 B
L
·
B
·
T manga, B
Coeficiente de Bloque Bloque de Referencia eslora entre PP Lpp
L
L
W
calado,
T
                                             J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
      D
D
DC
=s
= vol =
=
r 1.025 P L
·
A
X manga, B
Coeficiente Prismático Prisma de Referencia
Área sección
eslora entre PP Lpp
L maestra,
A
ML
Coeficiente de la
W
Sección Maestra
A
M =C B B
·
TC P
                                                            C
=
calado, T
área máxima bajo WL,
A X
    M
      J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
   A W
L
· B
Coeficiente de Plano de Agua
Plano de Referencia
L
        área plano de agua,
A
W
C
= WP
   manga, B
              eslora entre PP Lpp
  J. Vergara

      Tipos de Buques
     Factores en el rendimiento relativo de buques
   s
A W
Coeficiente Prismático Vertical
       área plano de
A W
C
PV
=
·
T
     agua,
         calado,
T
L
Prisma de Referencia
    J. Vergara

     Tipos de Buques
    Coeficientes de forma y ratios típicos
   Ratios
•
      Eslora
-
manga
     •
Lpp / B calado
Eslora
Lpp / T
-
     •
Coeficientes
• • • •
Manga
B/T
-
calado
     Bloque C B
Sección maestra C
Prismático C P
Plano de agua C WP
  M
  Destructor
Carga
Remolc.
Fluvial
Pasajeros
 9.82 32.75 3.33
   6.92 16.82 2.43
   4.18 9.33 2.23
   9.67 29.00 3.00
   8.38 26.25 3.14
     0.52 0.83 0.62 0.74
   0.64 0.96 0.66 0.76
     0.58 0.89 0.65 0.80
    0.87 0.99 0.88 0.91
   0.59
0.95 0.62
0.72
                              Otras relaciones usadas son el coeficiente volumétrico ( -
6
3
y la razón desplazamiento
-
eslora (10
·
D
/L
).
D
3 / L
)
        J. Vergara

      Tipos de Buques
     Centros y lugares geométricos del buque
    Varios atributos de diseño importan a la
flotabili
     dad,
que
comportamiento
de
- , antes
estabilidad
marinero
y
        los
, aspectos
energía
propulsión
.
    ●
●
●
● ●
Centro de Boyantes
(B):
Posición longitudinal de la LCB y posición vertical de la VCB.
     Posición longitudinal de la LCG y posición vertical de la VCG.
Centro de Gravedad
(G):
     Centro de Flotación
en el plano de flotación.
(F):
Posición longitudinal de la LCF
      Metacentro Transversal Metacentro Longitudinal
(MT): (ML):
Posición vertical de KM Posición vertical de KM
. T
. L
         J. Vergara

      Tipos de Buques
     Centros y lugares geométricos del buque
    KG KB
= Posición C de Gravedad = Posición C de Boyantez
K
= Keel
          W’
D
O
G B
K
L’
        Nota
= para conocimiento general de materias marítimas
D
     J. Vergara

      Tipos de Buques
     Centros y lugares geométricos del buque
      GM T
BM T
W W’
= altura metacéntrica = radio metacéntrico
= KM T
= KB
KM T
L’ KG L
KB
GZ (
= para conocimiento general de materias marítimas
f GM T
-
KG
         KM T
+ BM T
        DM GOZ
BB KD
T
          f
A WP
1
                        Nota
f
)
= brazo
adrizante
     J. Vergara

      Tipos de Buques
     Centros y lugares geométricos del buque
       GM L
BM L
= altura metacéntrica = radio metacéntrico
M
F
P M
L GM L
= KM L
= KB
-
KG
          KM L
+ BM L
      P PP
P PR
       LCF
W FFLCG
            W’ G L’
      BB
= para conocimiento general de materias marítimas
 Nota
1
L
     J. Vergara

      Potencia y Energía
       La potencia
está ligada a la sustentabilidad
 La sustentabilidad del sector marítimo está dada,
entre otros, por :
ii)
Las emisiones obedecen a una relación simple:
  i)
autonomía del recurso usado,
   mínimas emisiones en el ciclo de interés, y costos de servicio al alcance de los usuarios.
iii)
         a)
año) y c)
potencia emisiones
utilizada, b)
de la conversión de energía.
tiempo
de
operación
(i.e.
     E=P B
·
t
·
E
C [kgC/año]
        La clave
es
P B
Nuevas fuentes.
        J. Vergara

      Potencia y Energía
       Una prime
ra aproximación
       Podemos
aplicar
simples
relaciones
·
iniciales
.
    ● ● ● ●
Luego
Potencia = Fuerza
Velocidad ~ Resistencia
·
   Fuerza
Resistencia ~ V
Superficie mojada ~ (Volumen
     2
Superficie mojada
   s
)
2/3
    P
=
k
·
V
·
(
s
)
3
2/3
   :
de
Coeficiente Almirantazgo
       Confiable
buques
comparar proporciones
para
   de
=
  J. Vergara

      Potencia y Energía
       La potencia
está ligada a la sustentabilidad
          Potencia
al
freno
(P B
) es
aquella
que , etc.)
entrega
        (motor, de
rotación
Potencia
la de
fuente deducir
turbina
transmisión
hidrodinámica
efectiva
de las
poder pérdicas
,
luego descansos
,
             casco,
,
, etc.
      ), ; conduce a P
(P E
es la que
al
los
demanda
mover
         al cia
por componentes
buque propulsiva
dividirla
   (de
B todos
- ):
la
eficien
         P B
=P E
/
h
P [N
·
m/s ó W]
       J. Vergara

      Potencia y Energía
       Nomenclatura de las potencias en la propulsi
ón
  Siguiendo la dirección desde la hélice hacia la proa por el eje propulsor, según la posición en que se mida se tiene:
      Potencia Efectiva (EHP ó
vencer la resistencia (remolcar el buque).
P Potencia de Empuje (THP ó
●
●
●
ga
E
): es aquella requerida para
       ): es el trabajo de empuje (T) realizado por la hélice, a su velocidad de avance.
P
P
        ●
●
P
S
Potencia Entregada (DHP ó
T D
- ): es aquella que entrega el eje y
w
): es aquella que se entre
   en la hélice propulsora.
    Potencia al Eje (SHP ó
que se mide en un torsiómetro (
    Q
·
S
) dentro del casco.
     Potencia al Freno (BHP ó
fuente de poder. Se mide con un dinamómetro en taller.
P
B
): es aquella que entrega la
    J. Vergara

      Potencia y Energía
      Tendencias de potencia vs velocidad de buques
      Nu
dos
Avión Comercial
Tren Auto Helo
    Porta contenedor
Porta
- Aviones
       UL
    Camión
Hidroplano SES
  Buque de Carga
Destructor
P S
   Ton.
CC LCC VLCC
Grueso = k
·
V, con V = Volumen espacio cerrado y k = k(V)
Ton. Grueso
       Velocidad máxima (nudos)
1000 500
100
50
10 5
                                 0.1 0.5 1
5 10
50100 500
Potencia Específica
  J. Vergara

      Potencia y Energía
      La potencia efectiva es la forma más primitiva
     La
que se requiere para
do la resistencia hidrodinámica y es casi cúbica.
La arquitectura naval realiza gran esfuerzo para estimar esta potencia (canales de prueba, CFD, series de Taylor, etc.). Según Froude, es igual a
la
potencia efectiva
(P E
en W y EHP en HP) es la
     remolcar el buque,
-
vencien
       resistencia
total por la velocidad del buque:
    P E
=R
·V
   T
S [N
·
m/s ó W]
    J. Vergara

      Potencia y Energía
       La partida es la resistencia h
idrodinámica
     es la fuerza (N) requerida para re un casco a cierta velocidad (m/s) (en mar
La
molcar
calma y sin interferencias del remolcador). Suele diferenciarse resistencia y resistencia del casco desnudo. Se reconocen varios tipos:
Resistencia
-
         R R R R
R
Friccional Corrientes de Eddy
Formación de Ola Viento
Dinámica (aceler.)
, F
, E
,
, V
El diseño será bueno,
       cuando R T
sea mínimo
                 R
=
R
R
, Residual
       O
T
R
F
+
R
E
+ R
O
+
R V
+
R
D
           , D
   J. Vergara

      Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
    Es el caso más simple, sin R
p
+ -
ni R V
  O
. En teoría, se
    da la
gido está en un fluido perfecto (μ
-
Paradoja de d‘Alembert
si el cuerpo sumer
    G
4 L30 2
=
0, K
=
x
∞
,
=
0).
                 ¥
              1
¥
5 4 3 2 1
                         J. Vergara

      Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
    Es el caso más simple, sin R
da la gido
Por Bernoulli, se generan fuerzas de presión su
ni R V
d‘Alembert
. En teoría, se
si el cuerpo
        Paradoja de
O
sumer
G
-
     está en un fluido perfecto (μ
=
0, K
=
∞
,
=
0).
  - periores a la hidrostática, que actúan normal al
  cuerpo que se mueve.
            ¥
5 4 3 2 1
                        J. Vergara

      Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
    Es el caso más simple, sin R
da la gido
ni R V
d‘Alembert
. En teoría, se
si el cuerpo
         Paradoja
de
O
sumer
G
-
       está en un fluido perfecto (μ
=
0, K
=
∞
,
=
0).
     Por Bernoulli, se generan fuerzas de presión su
- periores a la hidrostática, que actúan normal al
p
RX
AX p
     cuerpo que se mueve.
              ¥
5 4 3 2 1
                          J. Vergara

      Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
   Fuerzas de Presión   Se Igualan   Fuerzas de Presión
(comp. en x) μ = 0
      que movimiento
el
que
Asisten
Resisten
el movimiento
      p
AX p p p
   RX
     AX RX
             ¥
5 4 3 2 1
                                J. Vergara

     Potencia y Energía
      Resistencia h
idrodinámica de cuerpos sumergidos
   Las fuerzas
por fuerzas
Como no hay Resistencia, esta sesión no vale....
¡¡ Afortunadamente, para el profesor,
Resistentes
en “X” son compensadas
   Asistentes
à
la resistencia global = 0.
      à
μ
>
0 !!.
¥
5 4 3 2 1
                               J. Vergara

     Potencia y Energía
      Resistencia h
idrodinámica de cuerpos sumergidos
  Con viscosidad, el agua se adhiere localmente al casco. A proa no es marcado. Hacia popa aparece la zona de transición, y comienza a generarse una estela notable. La frontera de esta “perturbación” se llama “capa límite”. En un vehículo real:
    Estela
Capa límite
Punto de Transición
   ¥
       5
5
      4
4
      3
3
    2
2 1
               J. Vergara

      Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
   Desde la zona de transición, el momentum entre
-
  gado se traduce en resistencia friccional. Los de
- fectos producen separación, y corrientes de Eddy.
  El efecto combinado de la capa límite y de las co
- rrientes de Eddy anula a las fuerzas “asistentes”.
      Estela
Punto de Separación
Capa límite
Eddy Punto de Transición
    ¥
5 4 3 2 1
                        Eddy, vórtice, etc.
    J. Vergara

       Potencia y Energía
       Resistencia h
idrodinámica de cuerpos sumergidos
       Punto de Separación
perfiles de velocidad
=R
R
   E=R (+ R
   F App
¥
5 4 3 2 1
     )
    App
R
App
                       J. Vergara

      Potencia y Energía
      Resistencia h
idrodinámica en la superficie
 En la superficie, al cuerpo sumergido, se le suma el efecto de la superficie libre. La variación local de presión, se traduce en ondas y la interferencia
   entre éstas genera la ola característica. Se
can las demás resistencias restando más energía
modifi
-
   R resisten al viento
R
del buque
à
O
. La obra muerta y superestructura
     à
.
R V
     ER
R
OR V
                F
          J. Vergara

     Potencia y Energía
     Teoría y experimentación hidrodinámica
  Esta teoría data del S.XVIII para fluido ideal
d’Alembert
Se verificó que no había relación lineal entre
(Euler,
    (
- . La experimentación viene
, Bernoulli)
y del S.XIX para fluido vis
    coso
con los vapores oceánicos y el casco de acero.
Navier
y Stokes)
         (
)
a
a
tone
-
y resistencia para escalar sus 3 buques.
laje
L
V
).
, postuló:
3
2
Esto sirve a
Brunel
(
Froude
    A menos que la confiabilidad de los experimentos a escala sean enfáticamente desestimados, es inútil gastar recursos en ensayos de gran tamaño, y que además pueden dar mal
viamente
   resultados, salvo que los fundamentos sean aclarados pre
- mediante exhaustivos ensayos a escala (1868).
        J. Vergara

     Potencia y Energía
      Análisis Dimensional (teorema
p
)
  Este teorema
1915)
les. En este caso
tales, masa [M], longitud [L], tiempo [T], tempera
Q
(de J. Fourier 1888 y E. Buckingham
      relaciona “n
p
-
m” parámetros
adimensiona
     m=
las dimensiones
- -
fundamen -
     tura [
modelo y buque, y evaluar su rendimiento.
], y
n
= variables. Se permite relacionar el
    p
p
p
p
p
G(
ó
p
, 1
, 2
, 3
)=0 n
p
p
,... 4
p
m
-
    1
=
f
(
, 2
, 3
,... 4
p
m
-
) n
   J. Vergara

     Potencia y Energía
      Análisis Dimensional (teorema
p
)
  Por ejemplo:
depende de 6 variables: velocidad, tamaño, den
sidad
[M, L, T] resulta:
R
La resistencia propulsiva del buque
     , viscosidad, gravedad y presión (con
p
m
- :3
   , 1
, 2
p
,y 3
p
). 4
p
r
      T
=
f
(V, L,
, μ, g, p)
          Ra T
ó b
r
c ,μ
d ,g
V
,L
,
e ,
f p
a
   J. Vergara

     Potencia y Energía
      Análisis Dimensional (teorema
p
)
    Un método de solución es mediante “
repetición
”:
          p p
=V d
r r
1 2
-
a
b
Igualando expo nentes para cada
-
2 V
,L ,L
,
c ,
R μ
1 g
1 p
1
1
        e ,
r
         p p
f m
dimensión se lo gra lo siguiente:
3 4
k
=V h
, ,
=V n
, ,
,L q
r r
        =V
,L
, LV
t
·
=R
1
T,
        p 2pr
p
= 2g
           R
Tr
V
3
    2= L
f μ,g·L,
2
2
L
·
·
V
·
L L,
· =p
        ·
V
·
·
V 2p
· μr
2
=
p
r
r
·
V
    4
·
2 V
      J. Vergara

     Potencia y Energía
      Análisis Dimensional (teorema
p
)
     Con S (superficie mojada),
en vez de L
, queda:
T
(p es alta y el fluido está en
2
       R
1r 2
p
· L,V,p=C
Lr ·
fV 2ng
                ·
T=
S
·
V
·
V
2
    Si despreciamos
 4
contacto con el casco), las fuerzas
hidrodinámi
   -
  cas controlan la similitud.
Si hay separación por
  presión, habrá cavitación y pierde similitud.
· L,V=
·
â
Luego: C
=
fV ng
f
(
,F
)
                  T
R
L
E
D
  J. Vergara

      Potencia y Energía
      Similitud (hidro o aero)dinámica
 Para que el rendimiento de un buque a escala real (S) sea derivado desde un modelo a escala (M), se deben cumplir las siguientes condiciones:
    : Las razones dimensionales (ej. L/D, B/H, etc.) deben ser iguales.
Similitud Geométrica
●
●
dor tud
buque deben ser de magnitud correspondientes.
        Similitud Cinemática
: Los campos de flujo
alrede
magni
-
      del modelo y del buque deben ser de correspondientes y en igual dirección.
-
     ●
Similitud Dinámica
: Las fuerzas asociadas con el movimiento del flujo alrededor del modelo y del
       J. Vergara

      Potencia y Energía
      Similitud (hidro o aero)dinámica
    Por partes: si μ = 0, desaparece el efecto de
queda el efecto de ola y Eddy (R residual o R RR
y
):
Buques geométricamente
R
E
          R
·
Con μ
habrá fricción (R friccional o R F
       C
L= ·
R
V
f
(F
)
      R
=1r 2
S
fg
D
similares tendrán igual C R
2=
0 y lejos de la superficie y de olas, sólo
     ·
≠
V
. D
si se desplazan a igual F
      ):
       R
·
·
Buques geométricamente
similares tendrán igual C F
. E
FV
L
f
(
)
       C
F
=1r 2
S
·
2=
fn=
R
E
          V
si se desplazan a igual
R
     J. Vergara

     Potencia y Energía
     Similitud (hidro o aero)dinámica
 Froude observó que la olas del buque se debían a
la resistencia, pero no valían las relaciones cono
de “
Si
V
 -
V y largas a
pendían del tamaño del buque. Así adopta la idea
     cidas. Las olas (cortas a
-
/L M
S=l y, M
â
velocidades correspondientes
á
V) no de ”, de Reech.
l
S=L
ML
              ”M”
es buque, y con
MV
=L S
:
es modelo y
S=V
MV
”S”
                          g
=C
L C
·
2S RM =2=S
·
·
Sg
L
             R
V
·
RS 1
r
·
LD
S
·
          S
r
S M
S S==
l
            RS
RM 1 2
3 L
      R
S
·
MD
V
M
2S
M
·
M
            J. Vergara

      Potencia y Energía
      Consecuencias de la similitud hidrodinámica
       V Si
l
  =R
R=
fVV
= 20 y = 30 kn.:
=
           C
=l LSMV
      R
· 2V
    1r R
·
2g
S M
 ·
V
·
S
6.7 kn
.
           l
 l Si
Las condiciones de similitud no se satisfacen en 1 ensayo. Tampoco vale modificar la viscosidad en el modelo. Froude señala que hay que separar.
    FV
· LV
=V
/ MV
= 20 y = 30 kn.:
=
            C
2V
       F
=1r
·
2=
fnS
S M
       S
·
V
600 kn
.
              =R
T=
fV+
fV
· L=C
+C
        C
2
L2nR ·
F
         T1r
·
21g
     S
·
V
   J. Vergara

     Potencia y Energía
     Aproximación del modelo al buque
 Froude propone un procedimiento que se iguala la
resistencia residual del buque y modelo,
l
R
R
Ajustó su modelo con R
coeficientes geometrías. Así obtiene
   l
FM
2
Calcula la resistencia friccional del modelo
C
RS
=
C
RM
    l
Establece correspondencia dimensional (
). , mide la resistencia total del modelo
/
. basó sus experimentos en superficies
     l
V M
TM
Con
y calcula
,
        C
=2
·
R
r
·
S M
·
(V M
)
  TM
TM
      Froude
planas y un canal de pruebas del Almirantazgo.
      =
C
ƒ
/
·
S M
·
V M
n
, y obtuvo
     ƒ
FM
y n, para diferentes materiales y
 FM
=2
·
R
FM
r
·
S M
·
(V M
)
2
             J. Vergara

      Potencia y Energía
      Aproximación del modelo al buque (cont)
   l
Se obtiene el coeficiente residual del modelo:
C
      =C
l
    RM
TM
-
C
FM
=C
RS
       l
Con V
S
V M
, y con S
S
=
2 ·
S M
, se determina la
=l
resistencia residual del buque
·
Se estima la resistencia friccional del buque
usando los coeficientes de
algunos “
         R
R
l
3 .
=R
,
  RS
RM
·
   l
FS
     y n, y R de la ITTC.
Froude
= 0.00871+0.053/(8.8+L).
ƒ ƒ
FS
=
      ƒ
Hoy se aplica el coeficiente C
·
Se agrega la resistencia del oleaje y viento
S S
(V S
·
)
n
. Ejemplo de
   F
” de rugosidad del casco.
   l
R V
y
 allowances
         J. Vergara

      Potencia y Energía
      Modelos de resistencia friccional
   0.009
   C F
= 0.0586
2 A.T.T.C. (1947) )) Schoenherr‘32
       (
= 0.066
(log 10
R
•C
     C F
(log 10
R
E -
F 2.03)
2
2 Hughes (1948) I.T.T.C. (1957)
            C F
E = 0.075
    C
=
F
 ́
(1 + k )
·
C F
(log 10
R
E
-
2)
k = 0.0008
k = 0.0004
          0.008 F
    C
0.007
         0.006 0.005 0.004 0.003 0.002 0.001
0.000
10
9R 8 10
10 E 10
                                        5 10
6 10
7 10
   J. Vergara

      Potencia y Energía
      La resistencia por formación de ola pone un freno
  l
Esta es la resistencia responsable que hace que los buques sean tan lentos como los de hace un siglo atrás, a pesar de nuevos propulsores.
     Olas Divergentes de Popa
Olas Divergentes de Proa
      19o28’
        Olas Transversales
LW
       J. Vergara

      Potencia y Energía
      La resistencia por formación de ola pone un freno
      J. Vergara

      Potencia y Energía
      La resistencia por formación de ola pone un freno
   Las más importantes en R O
-
son las olas transver sales, que parecen adheridas al casco. El largo
    ) es función de la velocidad de onda o velocidad del buque (V). La relación entre éstas se obtiene de la velocidad de onda natural:
p
de onda (L W
              V=V
h pLW
g =W·
·
·
·
L2
     OLA
2
·
= 2.43
·L
[ nudos ]
tanh
notó que la condición más resistente
  W
   W.
sucede cuando L
Froude
    = L. F
= 0.45 es la
velocidad
  W (2.43) y es una “
D
    ” que sólo buques poderosos o esbeltos logran cruzar.
crítica
barrera económica
    J. Vergara

      Potencia y Energía
      Límite de velocidad de buques convencionales
   PE
s
“Barrera Económica”
 Casco en U (Carguero rápido)
    Casco lleno (OBO)
  Potencia p
or fricción
          (semi) sumergibles
F
D
=V
/g
·
L
       Potencia Específica (kW/ton)
50 40 30
20 10
00 0.3 0.6 0.9 1.2 1.5 FD
           J. Vergara

      Potencia y Energía
      Ensayos de resistencia por formación de ola
  La experimentación data del siglo antepasado. Algunos han medido la resistencia integrando la
presión local en el casco, o la energía en la este
   -
   la. Como respaldo teórico, el buque puede
simu
  larse
- vía fuentes y sumideros distribuidos en L.
  A bajas V la resistencia es puramente friccional
. A mayor velocidad hay un leve descenso, pero
  sobre la “velocidad crítica” la resistencia puede tener un exponente de 6 ó más.
El siguiente experimento teórico
     -
ma el perfil de la ola y las interferencias (5 comp).
práctico confir
-
  J. Vergara

      Potencia y Energía
      Ensayos de resistencia por formación de ola
   Casco de Prueba: L = 4.9 m, B = 0.46 m, T= 0.3 m, C
V
WL
 B
= 0.636
           Perturbación Simétrica
Sistema Proa Sistema Corte Proa Sistema Corte Popa Sistema Popa
Perfil Total
@V/ gL=0.26
#1 #2 #3 #4
0.1 m #5
0.05 m 0T
           Experimental
Teórico
                               J. Vergara

      Potencia y Energía
      Ensayos de resistencia por formación de ola
 Esta resistencia es de naturaleza muy compleja.
             R
=
C
•1•
r
S
V
 O
O
•
•
F D
= 0.26
2 2
 Olas Transversales
 Olas Divergentes
Zonas de Interferencia
           C
0.0035
0.0030 O 0.0025 0.0020 0.0015 0.0010 0.0005
0.0000
0.2 0.3 0.4
                                0.5 F
D
(~V) 0.6
     J. Vergara

      Potencia y Energía
          L
as
component
e
s configuran la resistencia total
                R T
=
C
•1•
r
S
V
  T
•
•
2 2
C
O
 C
F
      C
0.008
T
0.007 0.006 0.005 0.004 0.003 0.002 0.001 0.000
10
                                     5 10
10 9 10
6 10
7 10
8 10
R
 E
 J. Vergara

      Potencia y Energía
      Si se hace a partir del modelo
   1
0.008
T
0.007 0.006 0.005 0.004 0.003 0.002 0.001 0.000
10
Obtener similitud geométrica (L, B, T, y
)
vs nS
10 9 10
l Conocer condiciones de prueba (ej. To, nM
    ,..)
       C F
(I.T.T.C.)
 C V
= (1 +
0.0004 ) • C F
V S
•
L S
     R
ES
=
S
   n
        C
RM = C
TM
-
C
FM
-
C
AM =C
RS
     R TM
en canal
 C
RS
 V M
=V S
/l
 k
·
C F
ó
C A
 R
=
M
V M
EM
L M
n
    •
C
FM
@
R
EM
   C
FS
@
R
ES
      C
                                                                        5 10
6 10
7 10
8 10
R
  E
 J. Vergara

      Potencia y Energía
           Importancia
del canal de
prueba
en
la
predicción
   J. Vergara

     Potencia y Energía
           Canales
ayudan
a
predecir
resistencia
por
olas
  Naval Surface Warfare Center, Carderock Div. Maneuvering & Seakeeping Basin (MASK)
    J. Vergara

      Potencia y Energía
       Podemos dete
rminar la resistencia total
  La resistencia hidrodinámica total del buque R
puede obtenerse con el método
en la
, TS
o
     Froude
por el procedimiento recomendado por la
-
mejor
  -
ITTC, por la vía de coeficientes y correcciones según el caso, para las condiciones establecidas
aún
    Misión
y los requerimientos de cada buque:
                 R
=
C
·1r
·
·
DS 1r
·
      TS
TS
2 22
2
S
S
V
=
C
+
C
+C
+
C
+
C
V
S
S
RS
FS
AM
VS
S
S
·
   J. Vergara

      Potencia y Energía
       Podemos dete
rminar la resistencia de submarinos
   Los submarinos y sumergibles modernos (AIP nucleares) pasan escaso tiempo en superficie, y se optimizan para navegación submarina (casco
ó
, pocos apéndices, un eje, etc.), los
     tipo
que no experimentan resistencia de ola:
Albacore
                1 22
  R
=C
·
r
1 (C
+ k)
·
r
0.2
·
·
S
+
C
·
S
·
·
V
S
·
     TS
T
S
V
S
2
=
VCD
CD
VAP
AP
2
S
      AP = Apéndices
CD = Casco Desnudo
1.8 C
VCD
CD
C
0.0002 a 0.0004
·
S
        S
3 = 1.03 C 2 ·
p
   CD
P
L
(
·B + 3
(B)
)
· (I.T.T.C.)
·
B
  C
C FCDF
·
FCD LL
3
à
1
+
0.5
·
                      J. Vergara

      Potencia y Energía
       Podemos dete
rminar la resistencia de submarinos
    T 1.60 C
C
TO 1.40
1.20 1.00
0.80 0.60 0.40 0.20 0
0 2 4 6 8 10 12 14 L/B
0.84
C = 4·r·D P p·L·B2
 CT @ CP
0.76 0.68 0.60
Rugosidad
CRCD CP = 0.60
k = 0.0004
Casco Desnudo
B
L
                                                      J. Vergara

      Potencia y Energía
       Podemos dete
rminar la resistencia total
  Estimada la resistencia hidrodinámica total del buque, será posible evaluar la potencia efectiva.
P E
     =R
TS
·V S
     Se puede construir una curva de potencia efecti
va para diferentes velocidades y las correspon
dientes resistencias.
Luego, conociendo la eficiencia propulsiva, se podrá construir la curva de potencia al freno.
-
-
      J. Vergara

      Potencia y Energía
      Tenemos una idea general de propulsión
  Interesa llegar a la Potencia al Eje (o al Freno).
70
Potencia
50
MW
40
30
20
10
0
0 5 10 15 20 25 30
V (nudos)
   Efe
ctiva
Ineficien
cia Pro
P
S
pulsiva
      P
E
=
R T
•
V
  P
E
8000 T
          60
                                  J. Vergara

     Combustibles Marinos
      Consumo anual es
∽
300 M ton de combustible
 Rutas Internacionales
Principales Centros Logísticos
Rotterdam Amberes
Ruta Futura
      L.A./
Busan
L.Beach
Panamá
Gibraltar S.Arabia Algeciras
Hong Kong
         Consumo anual de Energía:
11.5 EJ
Singapúr
       J. Vergara

      Combustibles Marinos
        Emisiones anuales de
∽
1050
M ton de CO 2
  La mayoría de los puertos almacena MDO y HFO.
  La mitad del suministro se concentra en 10
El 99.9% del consumo de búnker marítimo es:
HFO MDO MGO VLSFO
LNG
.
hubs
    225 Mt/a (Internacional)
    65 Mt/a (Doméstico
   20 Mt/a (Servicios
12 Mt/a
Pesca)
- Guerra)
 -
     A la potencia de régimen, esos combustibles
⋅
SO x
⋅
-
emi
), aparte de
,
       ten unos 1050 MtCO 2
/ año (
P
h
F
 varios contaminantes, i.e.
S
NO x
.
           J. Vergara

      Conclusiones
     Tenemos una idea general de propulsión
  Repasamos el rápido camino en el desarrollo de
los buques y la evolución de sus
   formas
.
  Además, hemos
aplicación y formas, las cuales tienen algún grado
clasificado
los buques según su
   de impacto en la potencia y en la
Vimos un
sustentabilidad
.
      tencia
se corrige
luego
método
práctico
- casco. Si ésta
necesaria para propulsar
para proyectar la
po
   un
la eficiencia del tren de
    mediante podemos conocer
trans la potencia a instalar y
- tecnologías y combustibles.
   misión
, seleccionar
         J. Vergara

      Conclusiones
          Desde
los
datos
en
un Canal de
Pruebas
  Para dimensionar la demanda de combustible, para cierta misión de buque nuevo, debemos:
     ● Estimar la resistencia
● Obtener el coeficiente de resistencia
● Agregar
● Estimar
● Conseguir el
total de
un
C
modelo M
         ● Deducir
● Deducir un margen (
● Estimar la resistencia residual (
TM modelo
el coeficiente C
viscoso del
    ) AM
    C
un margen (C
el coeficiente viscoso del buque S
= RM
coeficiente de resistencia C
     ) AS
C
) RS
           ● Obtener resistencia y P
del
buque
TS
 E
 J. Vergara


Conclusiones

   Tenemos una idea general de propulsión
  En esta sesión nos hemos centrado en el
su
casco
y
   resistencia
al avance.
  Con la potencia de la fuente de poder, y estiman do una tasa de utilización del buque, será posible
- del buque y de la flota.
   determinar las
A continuación, veremos los
emisiones
   propulsores
, las
y algunas ideas para hacer más
 fuentes de poder
sustentable el transporte marítimo, con nuevas
    opciones tecnológicas, mejorando la
sustentabi
lidad
global y
aprovechar
mejor
- el medio marino.

Introducción
    H
ace 60 años
, se produjo una discontinuidad
    Julio 19 inexplorado:
49:
prueba el 1
De Havillan
d ingresa a un terreno
     er
.
jet comercial
Comet 1
          DH Ghost 50 Mk1 W
Gloster E28/39
-
1
   J. Vergara
      Introducción
     H
ace 60 años
, se produjo una discontinuidad
       Boeing anuncia
Boeing
cuando
dos
Mod 367 “
1954
sólo
De Havilland
más
80”
       un
interesa
se avión
 veces
Dash
grande
.
           Le
Douglas, con el DC
Caravelle
,y
siguen
-
Sud
8.
Aviation, con el
Y no habían clientes.
    J. Vergara

       Introducción
      H
ace 60 años
, se produjo una discontinuidad
       La
a
Fuerza
Aérea
derivar
Boeing
1957
de EUA,
una
autoriza
        versión
707
comercial
a Boeing
del KC
-
135.
      Lo demás es historia,....el Jet se
como
establecía transporte
el
Diseño
Dominante
en el
.
        J. Vergara

       Introducción
      H
ace 60 años
, se produjo una discontinuidad
          A
una
salvo
la IIGM, se
fracaso
comercial
Comet
pesar falla
del
del conocida
, entonces
        estructural
,
poco
por
,
        casos marcaba
buques tendencia
de
fractura
en nueva
Liberty
      una incorporaba
de .
    El Jet
Mayor Nuevas
:
comercial
       Nueva Nuevos
formas )
fluodinámica
 Q Q Q Q
nuevas
alas,
). peso).
( turbopropulsores
     motores
( propulsiva
y
       potencia aleaciones
(y procesos
menor .
        J. Vergara

       Introducción
      H
ace 60 años
, se produjo una discontinuidad
        El Jet
comercial
del
Comet, se traduce en un
al permitía
,a
pesar
retraso caso
mientras
     el erosiona
nuevo
Mayor Mayor Menor Aviones
se
estándar
de personas y cargas de mayor valor.
investigaba
      que
transporte :
marítimo
      Este
concepto
       alcance
debido
a problema
ruta en
 l l l l
altura tiempo
sus
. motores
.
directa
        sin
      de grandes
.
vuelo
     más
.
  J. Vergara

       Introducción
       Como resultado de
la innovación
aeronáuti
ca...
   Indiferenciada
Bajo (¢/kg)
Capacidad
Alta (95% del total) Baja (5% del tot
Alto valor
al)
 Tipo de Carga
   Costo de Transporte
 Alto ($/kg)
   Tiempo de Transporte
 Lento (días) Rápi
do (horas)
  Velocidad de Transito
  Baja (20 kn) Alta (500 kn)
    Autonomía
 Alta (14000 mn) Media
 Baja (días) Alta
(6000 mn)
 Puntualidad
(minutos)
 Escasos
Pasajeros
Much
os
                             J. Vergara

       Introducción
      Efecto del
Covid
en el transporte marítimo
   9800
9600
9400
9200
9000
8800
8600
8400
 0
Recaladas semanales de Portacontenedores
6 8 4
  Promedio móvil 4 semanas
 2019
2020
  9%
max
 semanas
                                                           0 2
10 12 14 16 18 20 22 24 26 28 30
 J. Vergara

       Introducción
      Efecto del
Covid
en el transporte marítimo
 El principal impacto ha sido el “efecto acordeón” por la menor producción y luego recuperación o ajuste parcial, y sus efectos en el transporte de crudo y combustibles refinados.
      Por otro lado, se prevén exigencias por un
porte sin emisiones (sin cambios en la conducta
trans
-
   de viajes de las personas e intercambio comer
- ) que coincide con pocas medidas efectivas
  cial
de mitigación. Surgirán impuestos y por ahora el sector debate el uso de combustibles pesados
      con
scrubbers
de azufre vs diésel
más limpio.
  J. Vergara

       Introducción
     El mer
cado actual abre una nueva oportunidad
 3.5
Precio
Ejemplo Transatlántico Ref: MGI – Fedex
   Capacidad Valor Carga Puntualidad Velocidad
       Capacidad Valor Carga Puntualidad Velocidad
  Na
Avan
Capacidad Valor Carga Puntualidad Velocidad
    ves
zadas Avanzadas
OPORTUNIDAD
Aeronaves
                de Transporte
3.0 2.5
   US $/kg
2.0 1.5 1.0 0.5
0
323028262422201816141210 8 6 4 2 0
             Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

       Introducción
      Q
ue puede aprovechar
la industria naviera
 3.5
Precio
Ejemplo Transatlántico
    Airbus A3
80 Freight Boeing B747 8F
 Propulsores Avanzados
 Turbinas ICR, UDF, Me
- jores Alas BWB, matls.
 CargoLifter T160, WIG?
Na
Avan
zadas OPORTUNIDAD
Aeronaves
 ves
Avanzadas
              de Transporte
3.0 2.5
      US $/kg
2.0 1.5 1.0 0.5
0
323028262422201816141210 8 6 4 2 0
              Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

       Introducción
    El presente desafío sectorial es sugerir respuestas
 3.5
Precio
 Ejemplo Transatlántico
   ¿Qué tecnologías hay disponibles?
 ¿
a
¿
Cuáles
serían
mediano
viables
plazo
cuáles
   Avanzada
?, y
sus
efectos
s
OPORTUNIDAD
?
 ¿Que otras trabas hay?
Naves
Aeronaves
  Avanzadas
             de Transporte
3.0 2.5
         US $/kg
2.0 1.5 1.0 0.5
0
323028262422201816141210 8 6 4 2 0
                 Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

      Introducción
   En un mundo tecnológicamente dinámico
 Aunque el desarrollo continuo de la tecnología da poco respiro a los desarrolladores que compiten,
  es posible configurar
nuevos vehículos
marítimos
    sustentables,
que superen las limitaciones
técni
-
      -
El sector marítimo puede contribuir a un mundo
cas
-
y los estándares vigentes
sus dimensiones ambiental, social y económica.
y
(sustentación), nuevos
anticipables
en
    mejor con nuevos
cascos
      , nuevos
formas de utilización; incluso recuperar algunos de los mercados anteriormente desplazados.
materiales
propulsores
, nuevos
usos
y
       J. Vergara

      Limitaciones Actuales
     Algunos factores claves del transporte marítimo
     Mínima
Resistencia
al Avance
MISIÓN
“CARGA”
Combustibles
Ajuste
Máxima
Eficiencia
Propulsiva
                  Hidrodinámica y Diseño del
Casco
Hidrodinámica y Diseño del
Propulsor
     J. Vergara

      Limitaciones Actuales
     Opciones para mejorar la sustentabilidad marítima
  Alternativas de gestión, tecnológicas y recursos:
   ●
Velocidad, integración modal, optimización de ruta, rotación rápida en puerto,
Refinamiento de la Misión:
   multi
-
predicción climática, etc.
         ●
cambio , etc.
, , etc.
Reducción
de
de la Resistencia
sustentación
:
burbujas
Nuevas perfiles
:
formas
,
,
           ,
polímeros
y
,
bulbos
          ● ●
aletas
Aumentos
Nuevos
bio
Mejor
Propulsivos
Velas
,
cometas
, etc.
(
, uranio
          Combustibles
-
:
fósil
torio
Hidrógeno
), LNG,
       combustibles,
CCS,
synfuels
        ●
Nuevas
selección
de
Propulsores
(CRP, WJ, SP, EM), corrector de
estela
:
hélices , etc.
    J. Vergara

      Limitaciones Actuales
      Refinamiento de la Misión
 La opción más eficiente es la de reducir el tiempo
   en puerto. Surge el concepto de
Puertos Ágiles
Un puerto ágil acomoda los sistemas flexibles de carga tradicional y avanzada. A veces considera sistemas y preparativos para un apresto militar.
.
       Convencional
Agiles
Ro-Ro Alta velocidad
Multimodal FastShip
    J. Vergara

      Limitaciones Actuales
      Refinamiento de la Misión
         J. Vergara

      Limitaciones Actuales
      La resistencia es el factor más severo
         M
MISIÓN
“CARGA”
Máxima
Eficiencia
Propulsiva
M
í
ín
ni
i
m
m
a
a
    R
stencia Avance
Re
es
s
i
is
     a
al
l
       Debido experimenta
, el
Buque
a la
una
viscosidad Resistencia a
del
medio
      su
descomponer
RT =RF +RO +RE +RV +RA +...
  La Resistencia se
,
. según
Avance
       el
fenómeno
que
puede opera:
  J. Vergara

      Limitaciones Actuales
         Al
go
podemos hacer por la resistencia fricciona
l
   0.009
            R
=
C
•1•
r
S
V
 F
F
•
•
2 2
     C
=
0.075 2
I.T.T.C. (1957)
  F
10
-
2)
  C
=
F
 ́
(log
R
E
(1 + k )
·
C F
k = 0.0008
k = 0.0004
        0.008 F
  C
0.007
      0.006 0.005 0.004 0.003 0.002 0.001
0.000
10
9R 8 10
10 E 10
                                         5 10
6 10
7 10
   J. Vergara

      Limitaciones Actuales
       L
a resistencia propulsiva puede reducirse
         Si no se
aceptara
casco
  carga
sería
resta moderar
un tecnología
fino
clásica
( para
capacidad la
       de Resistencia
), l
a
usar
       modificadores
. suele
de
ola
           El proa vante ganar
es generarán
modificador
clásico dispositivos
el
fuerzas
Bulbo
,
de le
que
-
ir para
a
        .
Otros
,
un
           como
una
cuña
espejo o
en el velocidad
de economía
popa
.
,
         poco
obs
de velocidad
más
          No
tante
, la
se porque
ve moja
Estado de Mar
y
el más
reducida
por superficie
              somete
las
(S.S.) estructuras
a
altas
aceleraciones
.
 J. Vergara

      Limitaciones Actuales
       El mal tiempo agudiza el efecto de la resiste
ncia
 Afecta la Puntualidad (imagen) del Transporte
9 S.S. 8
(WMO)
7
6
5
4
3 2
    Velocidad Máxima
Buques Menores
Buques Medianos
vs S.S.
   Velocidad Limitada por Estado de Mar
Velocidad Limitada por Potencia
Buques Mayores
                                            0 5 10 15 20 25 30 V (nudos)
   J. Vergara

      Limitaciones Actuales
         Aparte de degradación,
pérdidas
y
atrasos
      One
Apus
8/12/2020
  Ver Videos
  J. Vergara

      Limitaciones Actuales
       Este
comportamiento puede mejorarse un poco
   Comportamiento Marinero
del Queen Victoria.
  L: 294 m, 89500 t, 63 MW, 24
kn En San Antonio, V Región
    J. Vergara

      Limitaciones Actuales
       El mal tiempo agudiza el efecto de la resiste
ncia
  Aumentos de Potencia Efectiva:
70
Potencia
50
MW
40
30
20
10
0
0 5 10 15 20 25 30
V (nudos)
  P
E
 ́
P
E
         60
                              J. Vergara

      Limitaciones Actuales
       Este
comportamiento puede mejorarse un poco
    El
nuevos cascos, i.e mayor
(eslora principalmente) y usando un forma más
Comportamiento Marinero
puede aliviarse
con
  superficie de flotación
   llena, en especial en la proa,
resistencia
Típicamente, se enfrenta el mal tiempo a menor velocidad y cambiando el rumbo, que lo retrasa.
      , y mediante
comprometiendo la
abanico
y
francobordo
.
        ,s
resistencia y comportamiento marinero para dar un servicio rápido, puntual y confiable.
Entonces
e
ap
recia una tramp
a viciosa entre
    J. Vergara

     Limitaciones Actuales
     En resumen, ha sido superada por la aviación
 Avión
Buque
      Fluido
- dinámica
 Superó su “R
al volar alto, y su “R con borde de ala en V.
” y “R O
  F
E
” ”
 Atrapado por R (viscosidad), y por R (su propia ola).
  F
E
,
yR
O
  Propulsor
 Desplazó el límite de eficiencia de hélices usando ventiladores.
 Restringido por
cavi
- y por el tamaño
tación
máximo de la hélice.
  Motores
 Superó la pérdida de succión del motor de pistones con turbinas.
 Enfocado en motores Diesel, privilegiando costo sobre el peso.
  Materiales
 Utiliza ampliamente el aluminio y aleaciones,
y ciertos
compositos
.
Utiliza acero suave, el HTS (alta resistencia) y rara vez el aluminio.
                                          J. Vergara

      Limitaciones Actuales
      Aparte, se sobreponen restricciones ambientales
      iente
buques de todo tipo. ¡Hay juicios sentenciados!.
Se acuñó la frase “
Se está apreciando una
presión
”, que implica
ambiental
crec
, que comienza a afectar el diseño de
medio
       Green
Stealth
   el
A primera vista parece una broma, pero merece
control de las firmas ambientales
.
   una consideración especial, pues esa
ha llegado, aunque sigamos usando petróleo (!).
Los arquitectos navales ya están incorporado medidas, ya que este fenómeno se va quedar.
tendencia
    J. Vergara

      Limitaciones Actuales
      Aparte, se sobreponen restricciones ambientales
      Green
Stealth
de:
), energía y altura de ola.
involucra la
mitigación
      l l l l l l
wake
Estela
wash
( Emisiones
    Acústicas
(aéreas y marina) . del Fondo Marino.
    Arrastre o
Emisión de
Aspiración y descarga de
Barrido
    Calor
al medio marino.
    Emisión de
Produce efectos importantes en la
.
a la Atmósfera, etc..
selección del
(máquina y propulsores).
Gases
Lastre
        sistema propulsivo
 J. Vergara

      Nuevas Oportunidades
      Evolución de costos del petróleo y del trabajo
  120
100
80
60
40
 20
0
1955 1965
1975 1985 1995
2005 2015 2025
 US$/bbl
CSub
Pr
Rusia
Paris
 US$/h
  Costo del Trabajo
   Valor del
Baja Re
Tiempo
sistencia
  Cambios en
 Propulsión
 Alta Velocidad
V
elocidad
y Eficiencia
   Concorde, Tu-144 RoRo Callaghan
Cos
Valor
to del Petróleo
de la Velocidad
Covid
  B 707
Año
                                         J. Vergara

      Nuevas Oportunidades
      Los costos anteriores tienen varios efectos
    1) A mayor valor del
Si vale más aún, compensado por el costo del
combustible, las
Tiempo
, mayor Velocidad.
     Personas
valoran los
ferries
  rápidos, los trenes rápidos y aviones locales
2) A mayor costo del
El transporte aéreo se encarece. La
Petróleo
. , menor Velocidad.
      Carga
de
   valor intermedio requiere
posible si el costo de inversión es estable.
,
medios alternativos
  J. Vergara

      Nuevas Oportunidades
          Proyección
de
emisiones
del sector
marítimo
    Transporte
Marítimo
Sector
0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8
 Emisiones por Clase de Buque
               Tasa de crecimiento Containerships
General Cargo
1,0%/y   1,5%/y
Passenger RoRo Naval
Emisiones 2010
GtC
    Crude Product
Bulk
2,0%/y
Oil
Tankers Tankers
Emisiones 2050
   Carriers
           Crude
Bulk
Product
Passenger RoRo Naval
Oil
Carriers Containerships
Tankers
Tankers General Cargo
        /y
                 J. Vergara

      Nuevas Oportunidades
          Proyección
de
emisiones
del sector
marítimo
    Transporte
Marítimo
Sector
0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8
 Emisiones por Clase de Buque
                    Crude Product
Refinamiento de la Misión Reducción de Resistencia Selección del Propulsor
Ajuste Propulsor Selección Fuente Poder Aumentos Propulsivos Nuevos Combustibles
Tasa de crecimiento Containerships
1,0%/y   1,5%/y
2,0%/y
Oil
Tankers Bulk Tankers
Carriers
General Cargo
Passenger
Emisiones 2050
          -
-
Fuente Poder
Casco
RoRo
Naval
Reduction
    Aumento
          Emisiones 2050 Reducidas
GtC
  2050 META 1,0%/y 1,5%/y
2,0%/y
/y
                  J. Vergara

      Nuevas Oportunidades
          Proyección
de
emisiones
del sector
marítimo
    Cuñas
de
Mitigación
 0.7 0.6 0.5 0.4 0.3 0.2 0.1
     GtC
/a rítimas de CO 2
   BAU (1,5%/a) 0.60
Selección de Propulsor WRE 1000 Optimización Propulsor/casco/motor
Selección de fuente de poder
   Aumentos Propulsivos à Nuevos combustibles (i.e. U, GN)
T=
+
5
°
C
 0.15
Emisiones Ma
Refinamiento de Misión
Reducción de Resistencia
Synfuels de base terres
tre, i.e. H2
WRE 450
+
     à
T=
2
°
C
                                   02000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100
            J. Vergara

      Nuevas Oportunidades
      El transporte es parte de un gran sistema
   2 1
1 2
  J. Vergara

      Nuevas Oportunidades
       El
“Valor” en las cadenas de transporte
 3.5
Precio
 Ejemplo Transatlántico
     gura Transf
Marítimo
Prioridad Estándar
   Hol
Aéreo
erencia Manejo- Terrestre Trav Aduana
     Priori
  Estándar
dad
           de Transporte
2.5
US $/kg
2.0 1.5 1.0 0.5
0
 3.0
      esía
           323028262422201816141210 8 6 4 2 0
Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

      Nuevas Oportunidades
            La estructura de
“
costos
” de
las
cadenas
difiere
 3.5
Precio
 ansporte
Mantención 4.8% Terminales 17.5%
5.3% Varios 4.1%
Mantención 5.1% 5.7% Varios Combustible
     Terminales 11.7% Contenedores 5.6%
34.6
       Tr
Terrestre
14.0% 10.6% Inversión 12.6% Gestión-Seguros
%
 Contenedores 18.1% 19.1% Gestión-Seguros 11.4% Transporte Terrestre
Combu
 19.7% Inversión
stible
                 de Transporte
2.5
US $/kg
2.0 1.5 1.0 0.5
0
3.0
                    323028262422201816141210 8 6 4 2 0
Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

      Nuevas Oportunidades
        La demanda se captura of
reciendo
mayor valor
 3.5
Precio
Ejemplo Transatlántico
  Para recuperar el posicionamiento
histórico, estas Naves A
vanzadas deben ofrecer un Mayor Valor.
 Velocidad M
35
nudos (0.5
ín
ima:
$/kg)
OPORTUNIDAD
           de Transporte
3.0 2.5
      US $/kg
2.0 1.5 1.0 0.5
0
323028262422201816141210 8 6 4 2 0
                Tiempo de Transito Puerta a Puerta (días)
  J. Vergara

      Nuevas Oportunidades
       La demanda se captura of
reciendo mayor valor
 3.5
Precio
 Ejemplo Transatlántico
        Hol
gura Transf
Marítimo
Aéreo
Prioridad Estándar
   erencia Manejo- Terrestre Trav Aduana
        Prioridad
     Estándar
           de Transporte
2.5
US $/kg
2.0 1.5 1.0 0.5
0
 3.0
      esía
        323028262422201816141210 8 6 4 2 0
Tiempo de Transito Puerta a Puerta (días)
   J. Vergara

      Nuevas Oportunidades
       El Factor de Transporte c
omo herramienta
     Para representar los atributos de d
culos y permitir su comparación, se puede utili
zar el
istintos
vehí
- -
     Factor de Transporte
Peso Total Velocidad de Diseño
(FT).
     FT =
W•V PS
     Potencia Nominal (propulsión y sustentación)
 J. Vergara

      Nuevas Oportunidades
       Explorando
en torno al Factor de Transporte
   Su Peso puede subdividirse en: a)
Máquinas W S
      , b)
Carga W C
Estructuras y
y c)
Combustible W
:
   F
 FT= W•V = WS•V + WC•V+ WF•V PS PS PS PS
         FTS + FTC + FTF
  J. Vergara

      Nuevas Oportunidades
       Explorando
en torno al Factor de Transporte
   Rearreglando
FTS
, se obtiene la siguiente expresión:
  FT=WC•V• WS+1+A•CEC PS WC
        FTC FTF
 J. Vergara

 Nuevas Oportunidades
Explorando
FTC 0.012
V
Ton 0.008
kW 0.006
0.004
0.002 0
0.010
Buques
en torno al Factor de Transporte
Vehículos rápidos
FTS + FTC V
0.7
s m
Aviones
0.01 0.2
FTS 0 2 4 6 8 10 12 14 FTC
J. Vergara

      Nuevas Oportunidades
Explorando
en torno al Factor de Transporte
       24
FTF
20
16 12 8 4 0
0.30 0.25 CEC = 0.35 kg/kWh 0.20 Aviones
Buques
CEC = 0.15 kg/kWh
0 2000 4000 6000 8000 10000 12000 14000 A (mn)
                                J. Vergara

      Desafíos Tecnológicos
         Los
desafíos
afloran
del Factor de
Transporte
 Del Factor de Transporte,
de: a)
    Misión
y b)
observamos Elementos
Diseño
Nuevos Propulsores
Nuevos Materiales
      FT =
C P •
P W
+ 1 + A•CEC
WC•V WS W •V• h
         PS C E
  Otra Sustentación
Nuevas Maquinarias
   J. Vergara

      Desafíos Tecnológicos
     El principal es superar las limitaciones intrínsecas
    Para recuperar el posicionamiento requiere transportar bienes a cierta
histórico
frecuencia
,
.
se
   ,
   y con un
doble requisito
1) a una velocidad >>
:
35 0.5 $/kg
kn
3.0 2.0 1.0
 US$/kg
 días
       2) a un c
La
y tencia
les
osto
Resistencia
<<
0 28 24 20 16 12 8 4 0
   al Avance de los buques clásicos
instalada
       las
Características
-
ta
condición
éstos ,
cumplir
de propulsores
sustentación
po
( materiales
esa
,
) son
        que
pueden
doble
.
,
no
-
          J. Vergara

      Desafíos Tecnológicos
      Un e
lemento clave es la sustentación
  FT=WC•V•hP• WS +1 +A•CEC PE WC
      Otra Sustentación
La incorporación de nuevas formas y conceptos
     hidrodinámicas permite reducir la
tiva
por
el
Potencia Efec
-
ni
  para alta velocidad, en especial la causada
     R
, cuidando que no afecte la
 O
estabilidad
.
   comportamiento marinero
 J. Vergara

     Desafíos Tecnológicos
    Modificando la sustentación de los vehículos
       aeroplano aeróstato
Vehículos
hidrostático
sumergible
       aerodinámico
aerostático
hidrodinámico
  híbrido
híbrido
                avión E.S. (SEA)
s
buque aerode
monocasco
multicasco
semi
catamaran SWATH
casco de
planeo
sumergido
     E.S. (SES)
lizador
(ACV) profundo convencional
hidroala
              total
rompeola
secante
                     J. Vergara

      Desafíos Tecnológicos
     Para superar la barrera del casco clásico
   PE
s
Casco en U (Carguero rápido)
“Barrera Económica”
    Casco “V” Casco Planeo (fragata) (lancha rápida)
         Casco lleno (OBO)
  Potencia p
SWATH (ferry)
     Hidroplano
or fricción
SES
          F
D
=V/ gL
  (semi) sumergibles
      Potencia Específica (kW/ton)
50 40 30
20 10
00 0.3 0.6 0.9 1.2 1.5 FD
                  J. Vergara

      Desafíos Tecnológicos
     Últimamente, han proliferado distintos vehículos
 327
120
FT
100 W•V 80
PS
60 40
     à Mínimo Costo e
“Norasia Sus
Sub. “Mitsu
  Indiferen- ciación
an”
bishi”
   “SL-7 Sealand”
CD =~V-2
SS United States
 subs
 Monocasco
 SeaLance
FastShip
CargoLifter
                                           20 Gotland
Aerostato
Taurus
InCat SES 60 80
àAlto Valor y Frecuencia de servicio
Aeroplanos
140 Velocidad
  Multicasco
     0
Hidroplanos
0 20 40
SES 100B
100 120
500
   J. Vergara

      Desafíos Tecnológicos
     Vehículos de Sustentación Híbrida
  Aerodinámica
  B
Ekranoplan (SEA)
PF, DD, SS, SWATH Preplaneo
Multicasco rompeola Semiplaneo
Carrera en planeo Planeo
LHHS
Hidrodinámica
-
747
-
400X
      Vehículo Magnus
Hidrostática
SES
                   -
Trimaran
        SWAACS SES
CargoLifter
Aerostática ACV HYACS
PHM
                       J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Hidrostática
    AD
HS
       AS
HD
 Casco: en V Veloc: 35 kn
DAD
Ruston Eslora: 112 m Manga: 16 m DWT: 450 tons Alstom
    -
HD (4wj)
     Ferry de Alta Velocidad
Gotland
           J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Hidrostática
    AD
HS
       AS
HD
 Casco: mono
R Veloc: 32 kn
  HD (4wj) Eslora: 250 m
GAG
-
â
 O
Manga: 33 m Cap: 1500 TEU GD, BIW, KMM
        BathMax 1500 (proa invertida)
Monocasco
(pasajeros)
     J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Hidrostática
    AD
HS
       AS
HD
 Casco: Circular Veloc: 25 kn Nuclear
Eslora: 112 m Manga: 22 m W: 48000 ton
       : 30000 ton Mitsubishi
W C
  Submarino de Carga (subártico)
Solo es un Proyecto
(¡ pero elimina R
      O
!)
 13 m
24 m
            180 m
                     J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Hidrostática
    AD
HS
       AS
HD
 Casco: SWATH
(~2 submarinos)
(
â
Eslora: 70 m Manga: 16 m W: 3500 tons JJMA
   á
Sea State y Potencia)
      SWATH Oceanográfico
S
         mall
W
T
-
ater plane
A
AGOS 19 Victorious
rea
T
H
ull
win
    J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Hidrodinámica
    AD HS
      AS
HD
  Casco: Foils Veloc: 42 kn
    HD (2wj) Eslora: 27 m
Manga: 9 m Cap: Limitada Boeing
GAG
-
    Ferry de Pasajeros HIDROALA (1974)
JetFoil
    J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Aerostática
    AD HS
      AS
HD
  Casco: metal Veloc: 60 kn
Helice
Eslora: 260 m Manga: 65 m Capac: 160 t Auton.: 5400 km CargoLIfter AG
    -
Diesel
    Aeróstato de Carga
CargoLifter 160T
           J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Aerodinámica
    AD
HS
       AS
HD
   Avión
de
: 500
Carga kn
   Veloc Turboventilador
  Eslora Manga Capac: 130 t
: 70 m : 60 m
     .: 10000 km Boeing, Airbus
Aut
 Aeroplano de Carga
Boeing 747
    -
400 SX
   J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
       AS
HD
 Casco: Cat Veloc: 42 kn
    HD (4wj) Eslora: 124 m
Manga: 40 m Cap: 1500 pp + 375 autos
Kvaerner M.M.
GAG
-
     Ferry Catamarán (Multicasco)
Ro
Stena HSS
   -
Ro
   J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
       AS
HD
 Casco: Slice Veloc: 30 kn Pods: 4
Eslora: 32 m Manga: 17 m Calado: 4.2 m Cap: 50 tons Lockheed Martin
      SLICE (Multicasco~SWATH)
      J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AS
HD
AD HS
        Casco: WP Cat
   Veloc: 40
-
50 kn HD (4wj)
   -
DAD
Eslora: 80 Manga: 26 m Cap: 900 pp +
  -
100 m
   200 autos InCat Tasmania
 Catamarán Rompeola (Multicasco)
Ro
              -
Ro
DevilCat
   J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
 Casco: AWP Cat
   Veloc: 50
-
60 kn HD (4wj)
   -
DAD
Eslora: 90 Manga: 26 m Cap: 1200 pp +
  -
120 m
   460 autos InCat Tasmania
 Catamarán Rompeola (Multicasco)
Ro
   -
Ro Asistido por Aleta
             J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
  Casco: TriCat
   Veloc: 50
  GAG
60 kn HD (6wj)
-
Eslora: 260 m
-
   Manga: 50 m Cap: 1200 pp +
 Trimarán (Multicasco)
          J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
 Casco: Trimarán Veloc: 45 kn
DAD
Eslora: 127 m Manga: 32m Cap: 700 ton G. Dynamics
    -
HD (4wj)
    Trimarán VHSST
Uss Independence
      J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
  Casco: planeo Veloc: 65 kn
    HD (3 wj) Eslora: 65 m
Manga: 13 m W: 1000 ton
GAG
-
     : 900 ton Auton: 4000 km
W F
  Monocasco de Planeo
Destriero
(record cruce transatlántico)
        J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
  Casco: SPMH Veloc: 40 kn
    HD (5 wj) Eslora: 262 m
Manga: 40 m W: 32.000 ton
GAG
-
     : 1432 TEU Auton: 4000 mn
W C
  Monocasco de Semìplaneo
FastShip
         J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
  Casco: ACV Veloc: 60 kn
DAD
Eslora: 29 m Manga: 12 m Rescate Aéreo control derrame Westland AE.
    -
AD
    Hovercraft
CH
   -
CCG Siyay
   J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
      AS
HD
  Casco: SES
   Veloc: 60
-
80 kn HD (2 wj)
   -
DAD
Eslora: 30 m Manga: 10 m Experimental Alto potencial
    Buque de Efecto Superficie
SES 100 B
   J. Vergara

      Desafíos Tecnológicos
       Vehículos de
S
ustentación Híbrida
    AD HS
       AS
HD
 Casco: SEA Veloc: 220 kn
   TP Enverg: 110 m
Altura: 6 m Alc: 16000 km
GAG
-
     Wing Boeing
-
-
Ground
In
 Avión Efecto de Superficie
Boeing Pelican
        J. Vergara

      Desafíos Tecnológicos
     Otro elemento clave es la fuente de poder
   FT=WC•V•hP• WS +1 +A•CEC PE WC
      Nuevas Maquinarias
Al mediano plazo se aprecian tres fuentes que
       superarán las limitaciones de
y
de ciertas plantas
Peso
,
Volumen
    Arreglo
y modo de
actuales (grandes motores Diesel y vapor) que no son aptos para alta velocidad.
Operación
   J. Vergara

      Desafíos Tecnológicos
       Sin
olvidar el consumo de la fuente de poder
     El vela y el vapor son historia. Hoy,
de Poder más eficientes son los Motores Diesel
lentos, por
l
Costo y Consumo de Combustible
as Fu
entes
    .
son grandes y pesados, además de
 No obstante,
lentas en reaccionar. Hay varias alternativas.
     kg/kW
Peso con F.O.
Diesel Lento
30 60
 Diesel Medio
10 53
 Turbogas
3 55
 GTMHR
15 15
                    J. Vergara

      Desafíos Tecnológicos
      Creciente presión sobre las emisiones
 20
18
16
14
12
10
8
6
4
2
0
0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400
      NOx (g/kWh)
 Tier I
(2000)
Tier II (2011)
 Tier III (2016)
  rpm
                              J. Vergara

      Desafíos Tecnológicos
      Otro elemento clave es la fuente de poder
  Las siguientes plantas propulsoras encuentran una aplicación específica en los nuevos buques, algunos buscando más economía y otro hacerlo
más velozmente (recuperar te
1) 2) 3) 4) 4)
      rreno
del
avión
)
).
(5 a 50 MW S
:
    Motores Diesel Rápidos
Turbogas
(< 10 MW S
      Aeroderivativas
).
Combustibles
(
     Motores y TG con LNG o Bio
Sistemas Renovables
Reactores Livianos
-
     aporte
)
    (> 50 MW S
)
.
   J. Vergara

      Desafíos Tecnológicos
      Motores Diesel lentos, medios y rápidos
    L: 20
-
30 m
  ↤
Motor
L: 6
kg/kWh)
reductor,
Motor Diesel 2T LS
   ↧
50
3000 rpm
TG
-
7m
↧
10 MW (SFC: 0.22 kg/kWh)
Diesel 4T MS
          à
MW (SFC:
0.3
DFO
    à
   à
85 MW (SFC: 0.16 kg/kWh) à
Scrub
     80
100 rpm directo, 14L 1200 rpm
à
-
HFO (+
reductor, 20V ), ULSFO MGO, DFO
       J. Vergara

      Desafíos Tecnológicos
      Optimización del volumen de la carga
            Clase E
-
L:398m
-
B:56m
-
14470 TEU
-
1 x 80 MW 14
cil
           Clase 3E GII
-
L:399m
-
B:59m
-
20570 TEU
-
2 x 31 MW 7
cil
 New
Class
-
L:350m
-
B:54m
-
17000 TEU
–
1 dual x 55 MW
             J. Vergara

      Desafíos Tecnológicos
      La estrella para usos especiales es la
turbogas
                  Generador Eléctrico 2
Vapor
Turbovapor
Gases
Caldera recuperadora de calor
Solución
COGES
(GE)
30 (+5) MW
h
Generador Eléctrico 1
            Aire
Gases
max
= 50%
                                                     T
+
T
COMPR
TP
  u
u
r
r
b
b
o
og
ESOR
g
a
as
sL
L
M
M
2
2
5
5
0
00
0
                     J. Vergara

      Desafíos Tecnológicos
      La estrella para usos especiales es la
turbogas
   Muy compleja Solución
Baja confiabilidad ICR Gases (más fríos) (RR)
    Aire
25.2 MW
h
Generador Eléctrico
      max
= 43%
     Recuperador
 By-pass
   Inter- enfriador
CHP
VAN
Turbogas WR
        CLP
                     TP
     -
21
               J. Vergara

      Desafíos Tecnológicos
      La estrella para usos especiales es la
turbogas
      Turbogas Rolls Royce
-
Grumman WR
-
-
llison 601R Alstom,
Eurodyn, Etc..
21 (ICR)
        1
-
2 MW
5
10 MW
   OPRA, Turbomeca, Vosper, Etc..
A
            20
-
25 MW
   J. Vergara

      Desafíos Tecnológicos
     La tecnología hace más eficientes a las turbogas
  El efecto directo es favorable, aunque aumenta el peso y la complejidad relativa de la fuente de
poder. Mejora la
      densidad
de ésta y el
a Turbogas
FT F
.
  0.6 Kg 0.4
 Turboga
COGES
 s
ICR
Diesel
        CEC
Un
       eliminaría las fuentes de
poder de cruce ro, si las hay.
100 %Potencia
ICR
    kW•h
0.2 0
-
        0
20 40
60 80
  J. Vergara

      Desafíos Tecnológicos
          La WR
-
21 falla en el DDG1000
à
giro a la MT30
   J. Vergara

      Desafíos Tecnológicos
          Nuevos
combustibles
hacia
el
2050
    (blue) H HFO CCS   NH
-
-
-
N
B B B B B
B
F
B
   A
2L
Cà -
F
Cà
F
  -
-
    3L 4L
       MDO e
MGO CCS
Cà OHà
(electro)           CH
-
         CH
     LSFO VLSFO LNG Mix
3 Hà
b
w
(bio) C BECCS       n
m
             (
wind )deà
            -
(nuclear)
fc+e e à v e à
    J. Vergara

      Desafíos Tecnológicos
          Nuevos
combustibles
hacia
el
2050
         MCI 4T
FC Licuado
FC Comprimido
MCI 2T MCI 4T FC
MCI 4T MCI 2T FC
MCI 2T
MCI 4T
FC Comprimido
MCI 4T MCI 2T MCI TG
CCS tierra
CCS bio y otros BECCS
SMR PWR Uranio HTGR Uranio LMR Torio
Lícuado
      Reg
Reg Reg
Reg
Reg
                  Reg
Reg
       H 2
NH 3
CH 4
C
                   CH 3
OH H
         n
m
      - Uranio
CCS
BECCS
          2020 2025
2030 2035       2040 2045 2050
 J. Vergara

      Desafíos Tecnológicos
       La propulsión
n
uclear es apta para alta potencia
  La última tecnología de turbogas (ej. descansos magnéticos, regeneradores, compresores, etc.)
posibilita la Propulsión Nuclear (P S
La experiencia nuclear mercante ha sido escasa y contradictoria, siempre basada en el PWR.
    > 50 MW S
).
           La nueva carta es el
R
Su peso específico
lar al de un motor Diesel, pero sin considerar su combustible. Ahí está la principal ventaja.
GT
MHR
( odular refrigerado por
-
as (He) y elio).
T
urbo
G
      eactor
M
H
    (inferior al del PWR)
-
es simi
   J. Vergara

      Desafíos Tecnológicos
       La propulsión
n
uclear es apta para alta potencia
   0 10 20m
        NuFastShip
     FTF
1/2 FTS
1/2 FTC
              J. Vergara

      Desafíos Tecnológicos
       La propulsión
n
uclear es apta para alta potencia
            Reactor
Vasijas de Conversión de Potencia
Compresor AP
Compresor LP Turbina Compresor
                 Intercooler Enfriador
3600 rpm
3600 rpm
                                             Regenerador
Turbina Potencia
    J. Vergara

     Desafíos Tecnológicos
    Cada reactor puede alimentar varias VCP
 PCV Babor
Vasija Reactor
VCP Estribor
          oC MPa
  10 2.7
                         468 7.2
                       850 7.1
    3600 rpm
210 MWt
3600 rpm
520 2.7
       J. Vergara

     Desafíos Tecnológicos
    Una planta nuclear permitiría mayor carga útil
     FastShip Turbogas
D = 30,750 ton, SPMH
V = 37.5 kn crucero (42 max) Ps=250MWs (SF=90%)
D =7.25ton/TEU@L=70% F = ~ 4,800 tons
C = 1432 TEU (Carga = 34%)
$ 2421 / TEU
               FastShip Nuclear
D = 30,750 ton, SPMH
V = 37.5 kn crucero (42 max) Ps=250MWs (SF=87%)
D =7.25ton/TEU@L=70% F = Solo Take-home
C = 1763 TEU (Carga = 42%)
$ 2103 / TEU
                J. Vergara

     Desafíos Tecnológicos
        Conceptos híbridos d
e
energía solar
-
eólica
   Oceanbird
de
Wallenius
Marine
           J. Vergara

      Desafíos Tecnológicos
       Nuevas formas d
e
energía renovable
    P
tida
ropulsión
      por
-
asis cometa
 Propulsión por energía solar
   Dynarig
  J. Vergara

      Desafíos Tecnológicos
         Nuevas formas híbridas d
e
energía solar
-
eólica
 Podría incorporar:
    • Biodiésel
, total o %. • Hidrógeno, total o %.
    • Por cierto Nuclear...
Ecoship
de
PeaceBOAT
     J. Vergara

      Desafíos Tecnológicos
      Posibilidades marítimas con
electromovilidad
   LNG EM
  Propulsión eléctrica o híbrida
  J. Vergara

      Desafíos Tecnológicos
    Varios buques adoptando transmisión eléctrica
    Generación:
4 × 14,4 MW 12V46F 2 × 19,2 MW 16V46F
BT : 4 × 5.5 MW
    225300
V
Azipods : 3 × 20 MW
m
Transmisión
23
GT, 362 x 61 x 9.3
kn
.E
-
Propulsión: 60 MW Hotel : 36 MW Total : 96 MW
       J. Vergara

      Desafíos Tecnológicos
     Características de un sistema eléctrico pequeño
    DDß
Se genera a bordo
                  G
G 2x1.3MW
àDD 2x1.3MWG G
      6.6 kV 3
60 Hz
6.6 kV 3
1.5 MW 60 Hz
60 Hz
f
1.5 MW
3MW440V 3
f
    Bus
Bus
               2.1 MW
2.0 MW
¿
Bus
f
MM
2.1 MW 2.0 MW
      Eficiencia de transmisión?
   J. Vergara

      Desafíos Tecnológicos
     DD con propulsión y poder eléctrico integrado
 USS
Moonsor
(
Zumwalt
), DDG 1001, 9200 t (4.2 B$), 78 MW, 30
kn
        J. Vergara

      Desafíos Tecnológicos
       Otro elemento clave es el de lo
sm
ateriales
 Nuevos Materiales
Para maximizar la capacidad de carga, se debe reducir el peso del buque:
l Uso de aceros y procesos avanzados. l Diseños innovadores de alta inercia. l Uso extensivo del aluminio.
l A futuro, más titanio y
   FT=WC•V•hP• WS +1 +A•CEC PE WC
                compositos
.
  J. Vergara

      Desafíos Tecnológicos
      Rendimiento esperado de los buques avanzados
            50000
40000 WC
30000
(ton)
20000
A
10000 8000
   Tecnología Larg
  Tecnología Mediano Plazo
o Plazo
  Tecnología Actu
al
                        10000 6000 4000
          0
30 40 50 60 70 80V90
2000 0
  J. Vergara

      Propulsores Avanzados
      Algunos factores claves del transporte marítimo
   MISIÓN
      Mínima
Resistencia
al
M
“CARGA”
E
a
E
M
a
á
á
x
x
i
i
m
m
a
    f
f
i
i
c
c
i
i
e
e
n
n
c
c
i
i
a
    Avance
Propulsivca
del
.
      Un factor clave del
propulsor
También eficiencia
sistema
eficiencia ,...)
   hélice
la
del y la
es paletas
     ( importan
de eficiencia
transmisión
,
rueda
       la
de la
casco
       RT V PE h
P PB
 J. Vergara

      Propulsores Avanzados
      Algunos factores claves del transporte marítimo
       Mínima
Resistencia
al
Hidrodinámica
MISIÓN
“CARGA”
Ajuste
Máxima
Eficiencia
Propulsiva
Hidrodinámica
      Avance
               y
Diseño
Casco
del
y
Diseño
del
  Propulsor
  - dependiente. En la práctica, su utilidad depende
de la misión del vehículo y de la forma de casco. Se requiere pleno Ajuste entre casco y hélice.
El propulsor es un dispositivo aparentemente in
    J. Vergara

 Propulsores Avanzados
Otro elemento clave es el propulsor
Recordamos las limitaciones de las Hélices:
Erosión
(cavitación),
Tope dimensional Carga radial dispar Depresión local Fluctuación y pulsos Estela tangencial Movimiento y oleaje
ruido, vibración, ineficiencia
l l l l l l
J. Vergara

      Propulsores Avanzados
      Otro elemento clave es el propulsor
   FT=WC•V•hP• WS +1 +A•CEC PE WC
     Nuevos Propulsores
¿Qué hacemos para reducir las limitaciones?
l Aumentar los niveles de presión local l Evitar pérdida en la punta de las palas l Homogeneizar el flujo en el impulsor l Independizarse de la forma del casco
           J. Vergara

     Propulsores Avanzados
     Teoría simple y práctica en hélices
      La
je necesario para mover el buque.
empu
hélice
es un dispositivo que produce el
-
    z VzV
        AoS En t=0, ambas
        coordenadas coinciden
y
Q
o
   o
    o
=
-
w
•
t
    yx
        x
x
o oy
     Sistema de Coordenadas rotatorio en la Hélice
Sistema de Coordenadas moviéndose con el buque
     J. Vergara

      Propulsores Avanzados
      Teoría simple y práctica en hélices
   hélice de tornillo
es el dispositivo más común
La
entre otros mecanismos de empuje. Este puede
    visualizarse como un
movido por un eje, se “atornilla” en el agua.
hidroplano helicoidal
que,
    En este plano se produce una
cuya componente axial impulsa al buque.
, No obstante, esta hélice produce una componen
fuerza de levante
   - (radial) que no contribuye al
   te de
empuje del buque, existiendo algunos aparatos o dispositivos para convertirla en empuje.
torque lateral
    J. Vergara

      Propulsores Avanzados
      Teoría simple y práctica en hélices
  Algunos clases de “hélices” propulsoras de tipo tornillo son los siguientes:
Hélice de paso fijo (recta y curva).
Hélice de paso ajustable (recta y curva). Hélice de paso controlable (recta y curva). Hélice supercavitante y rompesuperficie. Hélice en ducto, u otro intensificador de flujo.
   ● ● ● ● ● ● ●
Hélice contra
Otras hélices (tandem, coaxial, estatores, etc).
            -
rotatoria, eje común o no.
   J. Vergara

      Propulsores Avanzados
      Teoría simple y práctica en hélices
   H
el empuje de diversas formas, algunos maduros
● ● ● ● ● ●
Cicloidal “Voith
Schneider” o “Kirsten
Boeing”.
ay otros tipos de “propulsores” que producen
   y otros en desarrollo experimental
- Paletas (lateral, de popa, fijas o ajustables).
:
     -
A chorro (bomba sumergida o superficial).
        Aerodinámico (turbohélice
Eólico (vela clásica, avanzada, rotores,....). Magnetohidrodinámico, MHD.
-
turbojet).
      J. Vergara

      Propulsores Avanzados
      Tipo y número de hélices
   es mejor para buques mercantes: En algunos buques mercantes y en la mayoría de
La
es más simple, eficiente y barato, y mejor estela.
hélice única
    los buques de guerra, se usan 2
+
hélices, por:
 a)
capaces de satisfacer la potencia total en una.
Exceso de carga, cavitación o falta de fuentes
   b)
etc..) o vibraciones transmitidas al casco.
Límite de calado (canales, puertos, diques,
   c) d)
Redundancia y confiabilidad operacional, y Maniobrabilidad a baja velocidad.
      J. Vergara

      Propulsores Avanzados
      Geometría de la hélice
    Area
Area
D
esarrollada, A
  D royectada, A
    P Area
P
        E
Area Círculo, A O
xpandida, A E
                           Cono o Núcleo
Palas o Aspas
D Diámetro
Hélice Giro Derecho
Destrógira
     Hélice Giro Izquierdo
     Bordes
Ataque Salida Levógira
     J. Vergara

      Propulsores Avanzados
      Geometría de la hélice
   Proyección Longitudinal
P Paso
Inclinación
P Proyección
c Transversal z z 2
.tL p
Curvatura
                      t
  c
  Pala Expandida
          Helicoide: @ 1.0 R
1.0 R 0.8 R 0.6 R 0.4 R 0.2 R
     @ 0.6 R @ 0.2 R
             DD
Razón Paso
Diámetro
hC
      y línea de OP
max
      t
  2
    -
Razones de Area
             P D
AR = A
PE AR = A
E AR = A OA
      PR =
D
DP
A OA
       O
         J. Vergara

      Propulsores Avanzados
      Hélices de paso ajustable, variable y/o controlable
  En casos en que se prevén golpes, las palas se
   funden por separado y se
cono robusto, permitiendo su recambio. Es
apernan
a un núcleo o
   - ajustar levemente el paso de la hélice.
posi
  ble
La hélice de paso controlable y reversible (CPP)
   es cada vez más usada, en especial para máqui
- nas no reversible (TG). Su núcleo, más grande,
 tiene un sistema hidráulico que cambia o revierte el paso de las palas. Su eficiencia es algo menor por este cono. Esta hélice posee varias ventajas, según los requerimientos operacionales.
     J. Vergara

      Propulsores Avanzados
      Hélices de paso controlable y reversible
  La unidireccionalidad permite hélices curvas y uso en rompehielos, con un borde “reforzado”.
               J. Vergara

      Propulsores Avanzados
      Hélices de paso controlable y reversible
     Ü
Ü
Ü
Ü
Ü
-
Irreversibilidad de la ma quinaria motriz (TG).
Uso de motores Diesel en operaciones de baja demanda de potencia.
        Operación de máx. ma niobrabilidad y paradas.
Uso de generación PTO, eléctrica acoplada.
Operación amplia de máx. empuje (remolcadores).
-
         J. Vergara

      Propulsores Avanzados
      Introducción al diseño de hélices
 Hay numerosas teorías que describen el principio
de funcionamiento de hélices. Entre otras, se den resumir en las siguientes:
Teoría del Momentum Simple en un “Disco”. Teoría de Impulso en una Hélice.
Teoría del Elemento Infinitesimal de una Pala. Teoría de Circulación (“línea de empuje”).
Hay formas más modernas: métodos numéricos de paneles en 2D/3D, fuentes y sumideros, y cada vez más la computación fluidodinámica (CFD)
 pue
-
   ● ● ● ●
             J. Vergara

      Propulsores Avanzados
      Introducción al diseño de hélices
  Normalmente hay dos propiedades de la hélice que se diseñan al mismo tiempo, buscando un óptimo ya que no se dan al mismo tiempo:
Máxima eficiencia. Mínima cavitación.
    ● ●
     cos de operación asociado. La segunda tiene que
-
La primera minimiza la potencia al freno y el
  to
  ver con la durabilidad de las hélices y la genera
- de ruido (más sensible en los submarinos y
  ción
buques de guerra).
  J. Vergara

      Propulsores Avanzados
       La eficiencia depende del diámetro
y giro
    La hélice clásica es más
velocidad y si posee pocas palas de baja cuerda.
WL
eficiente
si gira a baja
                          D
»
»
1x 120 rpm D WL
1.2x 90 rpm
                   D
»
1.6x 60 rpm D
»
1.8x 40 rpm
                                       J. Vergara

      Propulsores Avanzados
       La eficiencia depende del diámetro
y giro
    La
cavitación
, aparte de dañar la hélice, también
    degrada la eficiencia
.S
menos poca profundidad,
eficiencia
al mismo tiempo. Hay que optar.
e manifiesta con
   palas
y suele iniciarse en las puntas.
de cuerda pequeña, en Luego, en general es difícil lograr
     y
ara
alta velocidad, redunda de todos modos en cavi
á
   â
cavitación
 Como es difícil disipar la potencia necesaria p
        , sumado a
canía al casco, limita la potencia a la hélice.
- -
tación
, lo cual
vibraciones
por la cer
   J. Vergara

      Propulsores Avanzados
         Se Puede Elevar la
Eficiencia de
l Propulso
r
   Como opción
que obliga a otro arreglo propulsivo. Pero, esto
, se puede fraccionar potencia, lo
   afecta el flujo de agua
por la hélice (logra más
   torque que empuje) lo que reduce la
eficiencia
.
. estela mediante un ducto
       La
vando condiciona
vibraciones
las palas
se reducen
-
cavitación
          y
y las lectando
, lo
cur vez
def la
cual
a de la
       resistencia
mecánica
su hélice
 Se puede a modificar la
  Schneekluth, vanos, preestatores, hélices contra
rotatorias, u otras sofisticaciones.
-
   J. Vergara

      Propulsores Avanzados
         Se Puede Elevar la
Eficiencia de
l Propulso
r
       Estos elementos mejoran la forma de la estela,
la puntas. Las
tamaño
elevan
eficien
-
o potencia
    cia
reducen la carga en las
en
, se alejan del
hélices son grandes y no pueden crecer
casco y
.
                     J. Vergara

      Propulsores Avanzados
      Introducción al diseño de hélices
  Teoría del Momentum Simple en un “Disco”.
zz ww
y
                         xyx
Con el coeficien
te de empuje C
se obtiene: 2
      -T2
          ,C TT1r
= AT
=h
·
          A
·
2I
1+ 1+C
   O
V
  J. Vergara

      Propulsores Avanzados
       Introducción al diseño de hélices
  Teoría de Circulación en una pala.
    zz w
V A
G
r
bási
(r)
(r)
V t
w
          (r)
                 xyx
Este modelo permite un dimensionamiento
co
y
    de la forma de cada pala (paso).
-
     J. Vergara

      Propulsores Avanzados
      Introducción al diseño de hélices
  Teoría de paneles en propulsores avanzados
 J. Vergara

         Propulsores Avanzados
       Potencias, pérdidas y eficiencias
       Pérdidas
casco hélice soportes descansos
p
    cp
hp
sp
d
                    V
S
       Eficiencias
V A
p
p
    P
=2
B
Q B
·
·
n
·
            h h
SP
Q
/P
DP P
=P
=2
P S(
-
-
p
) ) ) )
·
·
n
·
       S
=P =P
S
B S
d
D
=2
p
           D
Q D(
p p
·
·
P
n V(
·
               B H
T E
/P
=T =R
s c
Potencias
· T
       h
/P
P
- TADh
          T
E
V
T
-
p
P
·
P S(
      J. Vergara

      Propulsores Avanzados
      Potencias, pérdidas y eficiencias
   Por encima, la hélice opera bajo el principio de una helicoi de que se atornilla en el agua. Debido al roce, sin embargo, resbala y avanza menos que el valor teórico. Gráficamente:
Algunas Relaciones
-
  ·
p VV
n Aw
    V
S
V
RA
   V
RR
             :
           1) 2)
Razón Resbala -s
miento Real
=V
A
-V
S 3) Fracción de Estela n p•n V V
          miento Aparente Razón Resbala
RA =1 ·
p VVw
=w=1
A
             -s
·
-AV n p•n
- SV
           R
= RR=1 p
S
  J. Vergara

      Propulsores Avanzados
      Potencias, pérdidas y eficiencias
  Es común utilizar 3 (+1) eficiencias:
    h h h
h
1)
2) 3)
, la eficiencia del S
tren de transmisión
     , la eficiencia de la B
,
hélice detrás del buque
,y
      , la eficiencia del H
casco
.
S : Shaft B : Behind H: Hull
           p
P2 D=Dh
·
·
n
· QPT
·
·
·
V
VPR
            =
T=Ah
p
S SSDDTA
E=T
   S
= QP2
= QPT
         P2
n
·
·
·
·
·
p
B
H
n
·
V
       J. Vergara

      Propulsores Avanzados
      Potencias, pérdidas y eficiencias
 Sin embargo, Q, el torque de la hélice en el casco,
laboratorio usando la similitud dimensional).
h
  , el torque obtenido en un ensayo en aguas abiertas (aquel que se puede obtener en el
difiere de Q O
      Aparece, entonces: 4)
rotativa.
p
p
, la eficiencia relativa R
         P2 = DO =
·
·
QQh PT
n
·
V =T=A
p
          h
P
O = O = B , donde
Q
h
·
   R
·
·
Dh DQ
O OP
         D2
DO 2
Q O
n
·
·
·
n
·
       J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
  La similitud dimensional también vale en hélices, permitiendo evaluar el desempeño de un modelo,
  en un casco de interés. Además, puede caracteri
- zarse el modelo de hélice en un ambiente de
  “aguas abiertas” (estela uniforme).
Acá, se usa variables parecidas al caso del casco: empuje y torque (T y Q), diámetro (D) y velocidad rotacional de la hélice (n) (m=3, ñ=8). Por ejemplo:
     T
a
r
r
=
f
(V A
, D, n,
, μ, g, p)
, ó
h ,p
              T
V A
,D
c ,
d ,μ
e ,g
a
f
b ,n
    J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
  Un método de solución es mediante “repetición”:
                p p p
c ,p
k ,
n
r r
1 T
μ g
  1 2 3
n
a ,D
b ,
r
=V A
q ,
,
=V A4
,D w
t
              =V A
e ,
r r
f5 ,p
,D h
,D
, Igualando exponentes :
d
=V A
x ,
v ,
1
          ,D =T
=V A
m
1
   p
p
p
     1
A4
 r r
·D AVTr
V V
2 D
2p 2μA=
  =
           L D5AAAA
V A
D
Dg
    = =r
f
p,
· ,,
D 2r
n
· 2V
                  g
V
·
=pr
V
2 D
2μV
V
   p
2 2A
   3
A
V
1 n
1 p
      J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
   p
p
no hay cavitación.
mismo ensayo por inconsistencia de velocidades.
p
p
  1 3
es el Coeficiente de Empuje C
, y dependerá de T
    (equivale a F D
3 2
(Coeficiente de Avance J) si
p
)y
p
Pero,
do que la superficie expuesta de la hélice es poca (la velocidad tangencial de la pala no es grande).
       y
p
3
no pueden usarse en el
        2
que corresponde a R
, no es relevante da F
-
       =T=
D,n
· fg
·
D
          1
C
T
1r
V
2 D
2V
2V
=
2AAA
              J. Vergara

      Propulsores Avanzados
     Análisis dimensional en hélices
  De igual forma, para el torque Q, se obtiene otro
p
p
        parámetro relacionado, i.e.
=V A
,D
1
, con
s ,
r
u ,Q
 6
r
  el cual se obtiene el Coeficiente de Torque C Q
: =Q
        6
C
Q
1r
V
2 D
=
2A
 3
la real, y si
l
       es la hélice modelo y
S
, 0.5
l
Si
se tienen las siguientes relaciones,
M
=D entre otras:
/D S
M
           n S
l
=n
l
=V M
l
·
l
·
0.5 V
-
M AS
    T S
=T M
·
3Q S
=Q M
·
4P TS
=P TM
·
3.5
                  J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
       yC
coeficientes resultan:
K Ar
=Q
Q VJK
= 0 (Bo
=J
. Los
C
llard Pull), estos se hacen infinito. Como J esto se resuelve usando “n•D” en vez de V
-
    Q
T
tienen V A
en el denominador. Si V A
, S
                     V
n
T
=
·
T
2 n
·
M A
D
  J=
PT
=
4
  ·
DK
           ·
5
T Q
r
·
2 n
·
D
         h
O
T=A=·
      P
DO 2
p
·
·
Q O2
pK
n
    J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
  Se han construido curvas adimensionales en flujo uniforme
  1.0 0.8
K
0.6 0.4 0.2 0.0
 4:100
 K
Q
 P/D = 0.6
-
1.2
 h
O
 K
T
K
T
         10K
 Q
h
       O
      T
                         0 0.2
0.4 0.6
0.8 J 1.0 1.2
 J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
 Los coeficientes de empuje y de torque se pueden modelar con funciones matemáticas simples (polinomio logarítmico), que se obtienen de un modelo regresivo múltiple:
        T
2
i S= 3 9 4=k
         k i,2
i,1
⋅
k i,3
⋅
k i,4
⋅
k i,5
⋅
      K
T
=r
i=1
EAR
z
⋅
J
⋅
⋅
⋅
(P/D)
Q jS=47 k k k k
      ⋅
n
⋅
D
                       K
5=k j,1
J
j,2
(P/D)
j,3
EAR
j,4
z
j,5
    Q
=r
j=1
  ⋅
n
⋅
D
2 J⋅K
Rangos de validez: z de 3 a 7, EAR de 0.30 a 1.05, y P/D de 0.5 a 1.4.
        h
pK
T
  O
=
2
   Q
   J. Vergara

      Propulsores Avanzados
      Análisis dimensional en hélices
 Vimos la paradoja de d’Alambert, en que las Eddy anulan a las fuerzas “asistentes”.
       Con la hélice en la popa, se acelera el flujo de
R=1 casco
t=D
-R
       T
T TT
       agua, y se
las fuerzas asistentes.
R
anulan más
T TyR desnudo, sin
 T apéndices
Empuje T
D
Resistencia
R
V S
       Luego, T en la hélice de
R
- be ser mayor que R. Esto
        se denomina
t Deducción de Empuje
:
.
Razón de
    T>R
T
T
       J. Vergara

      Propulsores Avanzados
      Finalmente, la eficiencia propulsiva es
   Conocido el problema de ajuste entre casco y
hé
-
    lice
h
h
·
h
h
, podemos evaluar
: H
       P PT
V
S=T
S = (1
t)
E=R H
·
=
VT
w
(1
-
t)
·
V
-
w
      T ·
        (1 TAS
V (1 Podemos construir una Eficiencia Propulsiva
·
-
)
-
)
·
      h
h
: P
           P
·
•V
•
S=R
•V
h
E=R
V
•
S=R •n•Q 2•
h h
h
              h
p
T
T
S
S= B
T
S
• /
h
•
    P
=
p
•n•Q T•V
H
S
B
          P 2• SSDA
                P
=
h
H
•
h
S
•
R
•
h
O
       eficiencia “buque”
“acoplada”
eficiencia “hélice”
  J. Vergara

      Propulsores Avanzados
       El propulsor
para alta velocidad es el Waterjet
 1.0 Eficiencia
0.9 0.8 0.7 0.6
0.50 1 2 3C4 T
 Zona
Zona
de Hélice
 de Waterjet
 Waterjet (ideal)
  Hélice de Tornillo (ideal)
   Hélice
Waterjet
        hj
         0
 J. Vergara

      Propulsores Avanzados
       El propulsor
para alta velocidad es el Waterjet
     Estator Impulsor
avante
    Ducto
Cuña
Cubo Deflector
Tobera
atrás
Cuña
                              Sin avance
                 J. Vergara

      Propulsores Avanzados
       El propulsor
para alta velocidad es el Waterjet
                                   Direccional Booster Direccional
      J. Vergara

      Propulsores Avanzados
       El propulsor
para alta velocidad es el Waterjet
                                 J. Vergara

      Propulsores Avanzados
       El propulsor
para alta velocidad es el Waterjet
                                 J. Vergara

      Propulsores Avanzados
      El propulsor
para alta velocidad es el Waterjet
         13 22
22 13
7
77
7
                            Maniobrabilidad
 J. Vergara

      Propulsores Avanzados
      Las resistencias y eficiencias integradas
        Mínima
Resistencia
MISIÓN “CARGA”
Máxima
Eficiencia
    al Avance
Propulsiva
          R
R
•
h
•
h
•
h
F
+R
R
+R V
+R
D
S
R
O
H
h
h
                 TP
S
P
      Ajuste
       Hidrodinámica y Diseño del
Casco
R
• V
h
S
Hidrodinámica y Diseño del
Propulsor
       T
    P
 J. Vergara

      Propulsores Avanzados
         La eficiencia propulsiva
y la
potencia al
eje
 La resistencia define la Potencia Efectiva (y al Eje)
70
Potencia
50
MW
40
30
20
10
0
0 5 10 15 20 25 30
V (nudos)
   Al
Efe
Eje
ctiva
Ineficien
cia Pro
P
S
pulsiva
 V Sh
   P
=
R T
·
  P
      P
E
=
R T
•
V
  P
E
8000 T
          60
                                       J. Vergara

      Propulsores Avanzados
           La potencia al
eje
(o al
freno
) y las
máquinas
  Las motores tienen su campo
70
Potencia
50 40 30 20 10 0
    Un motor
(i.e. de 2)
P
S
Fouling
, SS 6,...
El mapa de
un motor
8000 T
              60
      MW
                       0
5 10 15 20 25 30 V (nudos)
  J. Vergara

      Propulsores Avanzados
           La potencia al
eje
(o al
freno
) y las
máquinas
  Las turbinas a gas también tienen su campo
70
Potencia
50
MW
40
30
20
10
0
0 5 10 15 20 25 30
V (nudos)
    Una turbina
(i.e. de 2)
P
S
Fouling
, SS 6,...
El mapa de
una turbina
8000 T
             60
                             J. Vergara

     Conclusiones
      ¿Tiene
s
entido lo que se ha planteado?
   1) La tendencia en el transporte marítimo se so
valente
3)
ser
2) Estas ferries
-
a madurar
     porta en una
confluencia de tecnologías
equi
,
a la pasada revolución aeronáutica.
       tecnologías
comenzan
- en los
    de alta velocidad, luego estas se
tan en medios de mayor tamaño y finalmente ingresa el transporte marítimo de carga.
-
baja
y eficientes que otros modos.
adop
       Ciertos
buques
podrán exhibir
resistencia,
    más
rápidos
  J. Vergara

     Conclusiones
      ¿Tiene
s
entido lo que se ha planteado?
    4) Los buques rápidos serán
valor del tiempo, y porque la carga (aérea) se vuelve relativamente más lenta, a lo que se sumaría la baja intensidad energética de este segmento de transporte.
favorecidos
por el
       5)
el presente, con R/P ~45 años. A futuro, podrá
El transporte marítimo
depende del petróleo
en luego gas del carbón
      ampliar
GN
gas, con autonomías de
de
mayor R/
P,
siglos y
      y
rar combustibles sintéticos de diverso origen.
-
shale
explo
    J. Vergara

     Conclusiones
      ¿Tiene
s
entido lo que se ha planteado?
        6) sibles
irrever
-
Si las
(con o sin razones
emisiones se tornan científicas
relevantes ),
e
todas las
        plataformas deberán
opciones vistas
reducirlas
, mediante
(nuevos cascos
, nuevos combustibles, etc.).
las , propulsores,
     formas de uso
7) Lo anterior puede funcionar dentro de reglas
    de emisiones nacionales para el
Por la naturaleza internacional del transporte
IMO
.
cabotaje local
  8)
marítimo, deberán existir acuerdos vía
.
     J. Vergara

     Conclusiones
      ¿Tiene
s
entido lo que se ha planteado?
  9) Este sector es más simple que el aeronáutico
  porque las
son gestionables con diseño. Hay flexibilidad.
restricciones de peso y/o volumen
 10) Los grandes buques podrán utilizar sistemas
     nucleares
o
. Los costeros podrán usar
asistencia eólica
clásicos
lastrado en navegación y combustible). Los de mediana potencia y autonomía podrán usar
compactos
(minimiza el
     sistemas
baterías. Varios podrán tener
híbridos
   .
  J. Vergara

     Conclusiones
    La tecnología favorece a los buques rápidos
Avión
Buque Rápido
   Fluido
- dinámica
Borde de Ataque de Ala, en V.
Multicasco, SMPH, planeo, levante, ...
Propulsor
 Ventiladores o UDF.
Waterjet, y azipod para maniobra.
 Motores
 Turbogas (¿ICR?)
 Turbogas ICR y GTMHR
Materiales
Aleaciones Alumi nio, y compositos.
-
HTS y Aleaciones de Aluminio.
                         J. Vergara


Conclusiones

   Aunque la competencia tecnológica es dinámica


'''